In [1]:
#-*- coding: utf-8 -*-

In [2]:
from datetime import datetime,timedelta
import numpy as np
import json
import requests
import pandas as pd

# import urllib.request
import os
import tarfile
np.random.seed(10)

data_path = '/home/charles/pythonwork/PythonProject_test01/data'

In [3]:
import regex as re

# * 字符串预处理模块，为分析器TimeGenerator提供相应的字符串预处理服务
class StringPreHandler:
    
    @classmethod
    def delKeyword(self, target, rules):
        """
        該方法刪除一字元串中所有匹配某一規則字串
        可用於清理一個字元串中的空白符和語氣助詞
        :param target: 待處理字元串
        :param rules: 刪除規則
        :return: 清理工作完成後的字元串
        """
        pattern = re.compile(rules)
        res = pattern.sub('', target)
        # print res
        return res


    @classmethod
    def numberTranslator(self, target):
        """
        該方法可以將字元串中所有的用漢字表示的數字轉化為用阿拉伯數字表示的數字
        如"這裡有一千兩百個人，六百零五個來自中國"可以轉化為
        "這裡有1200個人，605個來自中國"
        此外添加支持了部分不規則表達方法
        如兩萬零六百五可轉化為20650
        兩百一十四和兩百十四都可以轉化為214
        一六零加一五八可以轉化為160+158
        該方法目前支持的正確轉化範圍是0-99999999
        該功能模組具有良好的復用性
        :param target: 待轉化的字元串
        :return: 轉化完畢後的字元串
        """
        pattern = re.compile(u"[一二兩三四五六七八九123456789]萬[一二兩三四五六七八九123456789](?!(千|百|十))")
        match = pattern.finditer(target)
        for m in match:
            group = m.group()
            s = group.split(u"萬")
            s = filter(None, s)
            num = 0
            if len(s) == 2:
                num += wordToNumber(s[0]) * 10000 + wordToNumber(s[1]) * 1000
            target = pattern.sub(str(num), target, 1)

        pattern = re.compile(u"[一二兩三四五六七八九123456789]千[一二兩三四五六七八九123456789](?!(百|十))")
        match = pattern.finditer(target)
        for m in match:
            group = m.group()
            s = group.split(u"千")
            s = filter(None, s)
            num = 0
            if len(s) == 2:
                num += wordToNumber(s[0]) * 1000 + wordToNumber(s[1]) * 100
            target = pattern.sub(str(num), target, 1)

        pattern = re.compile(u"[一二兩三四五六七八九123456789]百[一二兩三四五六七八九123456789](?!十)")
        match = pattern.finditer(target)
        for m in match:
            group = m.group()
            s = group.split(u"百")
            s = filter(None, s)
            num = 0
            if len(s) == 2:
                num += self.wordToNumber(s[0]) * 100 + self.wordToNumber(s[1]) * 10
            target = pattern.sub(str(num), target, 1)
        
        pattern = re.compile(u"[零一二兩三四五六七八九]")
        match = pattern.finditer(target)
        for m in match:
            target = pattern.sub(str(self.wordToNumber(m.group())), target, 1)

        pattern = re.compile(u"(?<=(周|週|星期|禮拜))[末天日]")
        match = pattern.finditer(target)
        for m in match:
            target = pattern.sub(str(self.wordToNumber(m.group())), target, 1)

        pattern = re.compile(u"(?<!(周|週|星期|禮拜))0?[0-9]?十[0-9]?")
        match = pattern.finditer(target)
        for m in match:
            group = m.group()
            s = group.split(u"十")
            num = 0
            ten = self.strToInt(s[0])
            if ten == 0:
                ten = 1
            unit = self.strToInt(s[1])
            num = ten * 10 + unit
            target = pattern.sub(str(num), target, 1)

        pattern = re.compile(u"0?[1-9]百[0-9]?[0-9]?")
        match = pattern.finditer(target)
        for m in match:
            group = m.group()
            s = group.split(u"百")
            s = filter(None, s)
            num = 0
            if len(s) == 1:
                hundred = int(s[0])
                num += hundred * 100
            elif len(s) == 2:
                hundred = int(s[0])
                num += hundred * 100
                num += int(s[1])
            target = pattern.sub(str(num), target, 1)

        pattern = re.compile(u"0?[1-9]千[0-9]?[0-9]?[0-9]?")
        match = pattern.finditer(target)
        for m in match:
            group = m.group()
            s = group.split(u"千")
            s = filter(None, s)
            num = 0
            if len(s) == 1:
                thousand = int(s[0])
                num += thousand * 1000
            elif len(s) == 2:
                thousand = int(s[0])
                num += thousand * 1000
                num += int(s[1])
            target = pattern.sub(str(num), target, 1)

        pattern = re.compile(u"[0-9]+萬[0-9]?[0-9]?[0-9]?[0-9]?")
        match = pattern.finditer(target)
        for m in match:
            group = m.group()
            s = group.split(u"萬")
            s = filter(None, s)
            num = 0
            if len(s) == 1:
                tenthousand = int(s[0])
                num += tenthousand * 10000
            elif len(s) == 2:
                tenthousand = int(s[0])
                num += tenthousand * 10000
                num += int(s[1])
            target = pattern.sub(str(num), target, 1)

        return target

    @classmethod
    def wordToNumber(self, s):
        """
        方法numberTranslator的輔助方法，可將[零-九]正確翻譯為[0-9]
        :param s: 大寫數字
        :return: 對應的整形數，如果不是數字返回-1
        """
        if (s == u'零') or (s == '0'):
            return 0
        elif (s == u'一') or (s == '1'):
            return 1
        elif (s == u'二') or (s == u'兩') or (s == '2'):
            return 2
        elif (s == u'三') or (s == '3'):
            return 3
        elif (s == u'四') or (s == '4'):
            return 4
        elif (s == u'五') or (s == '5'):
            return 5
        elif (s == u'六') or (s == '6'):
            return 6
        elif (s == u'七') or (s == u'天') or (s == u'日') or (s == u'末') or (s == '7'):
            return 7
        elif (s == u'八') or (s == '8'):
            return 8
        elif (s == u'九') or (s == '9'):
            return 9
        else:
            return -1
    
    @classmethod
    def strToInt(self, s):
        try:
            res = int(s)
        except:
            res = 0
        return res

In [4]:
sp = StringPreHandler()

In [5]:
class TimePoint:
    def __init__(self):
        self.tunit = [-1, -1, -1, -1, -1, -1]

class RangeTimeEnum():
    day_break = 6  # 凌晨
    early_morning = 9  # 上班|早上|早晨|早間|晨間|今早|明早
    morning = 10  # 上午
    noon = 12  # (中午)|(午間)|(午休)
    afternoon = 15  # (下午)|(午後)
    night = 18  # 下班
    lateNight = 20  # 晚上|夜間|夜裡|今晚|明晚
    midNight = 23  # 深夜
    

In [6]:
import arrow
import copy
import json

# from TimePoint import TimePoint
# from RangeTimeEnum import RangeTimeEnum
# try:
#     from LunarSolarConverter.LunarSolarConverter import *
# except:
#     from LunarSolarConverter import *


# 時間語句分析
class TimeUnit:
    def __init__(self, exp_time, normalizer, contextTp):
        self.exp_time = exp_time
        self.normalizer = normalizer
        self.tp = TimePoint()
        self.tp_origin = contextTp
        self.isFirstTimeSolveContext = True
        self.isAllDayTime = True
        self.time = arrow.now()
        self.time_normalization()

    def time_normalization(self):
        self.norm_setyear()
        self.norm_setmonth()
        self.norm_setday()
        self.norm_setmonth_fuzzyday()
        self.norm_setBaseRelated()
        self.norm_setCurRelated()
        self.norm_sethour()
        self.norm_setminute()
        self.norm_setsecond()
        self.norm_setSpecial()
        self.norm_setSpanRelated()
        self.norm_setHoliday()
#         self.modifyTimeBase()
        self.tp_origin.tunit = copy.deepcopy(self.tp.tunit)

        # 判斷是時間點還是時間區間
        flag = True
        for i in range(0, 4):
            if self.tp.tunit[i] != -1:
                flag = False
        if flag:
            self.normalizer.isTimeSpan = True

        if self.normalizer.isTimeSpan:
            days = 0
            if self.tp.tunit[0] > 0:
                days += 365 * self.tp.tunit[0]
            if self.tp.tunit[1] > 0:
                days += 30 * self.tp.tunit[1]
            if self.tp.tunit[2] > 0:
                days += self.tp.tunit[2]
            tunit = self.tp.tunit
            for i in range(3, 6):
                if self.tp.tunit[i] < 0:
                    tunit[i] = 0
            seconds = tunit[3] * 3600 + tunit[4] * 60 + tunit[5]
            if seconds == 0 and days == 0:
                self.normalizer.invalidSpan = True
            self.normalizer.timeSpan = self.genSpan(days, seconds)
            return

        time_grid = self.normalizer.timeBase.split('-')
        tunitpointer = 5
        while tunitpointer >= 0 and self.tp.tunit[tunitpointer] < 0:
            tunitpointer -= 1
        for i in range(0, tunitpointer):
            if self.tp.tunit[i] < 0:
                self.tp.tunit[i] = int(time_grid[i])

        self.time = self.genTime(self.tp.tunit)

    def genSpan(self, days, seconds):
        day = int(seconds / (3600*24))
        h = int((seconds % (3600*24)) / 3600)
        m = int(((seconds % (3600*24)) % 3600) / 60)
        s = int(((seconds % (3600*24)) % 3600) % 60)
        return str(days+day) + ' days, ' + "%d:%02d:%02d" % (h, m, s)

    def genTime(self, tunit):
        time = arrow.get('1970-01-01 00:00:00')
        if tunit[0] > 0:
            time = time.replace(year=int(tunit[0]))
        if tunit[1] > 0:
            time = time.replace(month=tunit[1])
        if tunit[2] > 0:
            time = time.replace(day=tunit[2])
        if tunit[3] > 0:
            time = time.replace(hour=tunit[3])
        if tunit[4] > 0:
            time = time.replace(minute=tunit[4])
        if tunit[5] > 0:
            time = time.replace(second=tunit[5])
        return time

    def norm_setyear(self):
        """
        年-規範化方法--該方法識別時間表達式單元的年欄位
        :return:
        """
        # 一位數表示的年份
        rule = u"(?<![0-9])[0-9]{1}(?=年)"
        pattern = re.compile(rule)
        match = pattern.search(self.exp_time)
        if match is not None:
            self.normalizer.isTimeSpan = True
            year = int(match.group())
            self.tp.tunit[0] = year

        # 兩位數表示的年份
        rule = u"[0-9]{2}(?=年)"
        pattern = re.compile(rule)
        match = pattern.search(self.exp_time)
        if match is not None:
            year = int(match.group())
            self.tp.tunit[0] = year

        # 三位數表示的年份
        rule = u"(?<![0-9])[0-9]{3}(?=年)"
        pattern = re.compile(rule)
        match = pattern.search(self.exp_time)
        if match is not None:
            self.normalizer.isTimeSpan = True
            year = int(match.group())
            self.tp.tunit[0] = year

        # 四位數表示的年份
        rule = u"[0-9]{4}(?=年)"
        pattern = re.compile(rule)
        match = pattern.search(self.exp_time)
        if match is not None:
            year = int(match.group())
            self.tp.tunit[0] = year

    def norm_setmonth(self):
        """
        月-規範化方法--該方法識別時間表達式單元的月欄位
        :return:
        """
        rule = u"((10)|(11)|(12)|([1-9]))(?=月)"
        pattern = re.compile(rule)
        match = pattern.search(self.exp_time)
        if match is not None:
            self.tp.tunit[1] = int(match.group())
            # 處理傾向於未來時間的情況
            self.preferFuture(1)

    def norm_setmonth_fuzzyday(self):
        """
        月-日 兼容模糊寫法：該方法識別時間表達式單元的月、日欄位
        :return:
        """
        rule = u"((10)|(11)|(12)|([1-9]))(月|\\.|\\-)([0-3][0-9]|[1-9])"
        pattern = re.compile(rule)
        match = pattern.search(self.exp_time)
        if match is not None:
            matchStr = match.group()
            p = re.compile(u"(月|\\.|\\-)")
            m = p.search(matchStr)
            if match is not None:
                splitIndex = m.start()
                month = matchStr[0: splitIndex]
                day = matchStr[splitIndex + 1:]
                self.tp.tunit[1] = int(month)
                self.tp.tunit[2] = int(day)
                # 處理傾向於未來時間的情況
                self.preferFuture(1)

    def norm_setday(self):
        """
        日-規範化方法：該方法識別時間表達式單元的日欄位
        :return:
        """
        rule = u"((?<!\\d))([0-3][0-9]|[1-9])(?=(日|號))"
        pattern = re.compile(rule)
        match = pattern.search(self.exp_time)
        if match is not None:
            self.tp.tunit[2] = int(match.group())
            # 處理傾向於未來時間的情況
            self.preferFuture(2)

    def norm_sethour(self):
        """
        時-規範化方法：該方法識別時間表達式單元的時欄位
        :return:
        """
        rule = u"(?<!(周|週|星期|禮拜))([0-2]?[0-9])(?=(點|時))"
        pattern = re.compile(rule)
        match = pattern.search(self.exp_time)
        
        if match is not None:
            self.tp.tunit[3] = int(match.group())
            # 處理傾向於未來時間的情況
            self.preferFuture(3)
            self.isAllDayTime = False


        # * 對關鍵字：早（包含早上/早晨/早間），上午，中午,午間,下午,午後,晚上,傍晚,晚間,晚,pm,PM的正確時間計算
        # * 規約：
        # * 1.中午/午間0-10點視為12-22點
        # * 2.下午/午後0-11點視為12-23點
        # * 3.晚上/傍晚/晚間/晚1-11點視為13-23點，12點視為0點
        # * 4.0-11點pm/PM視為12-23
    
        rule = u"上班"
        pattern = re.compile(rule)
        match = pattern.search(self.exp_time)
        if match is not None:
            if self.tp.tunit[3] == -1: 
                self.tp.tunit[3] = RangeTimeEnum.early_morning
                # 處理傾向於未來時間的情況
                self.preferFuture(3)
                self.isAllDayTime = False
                
        rule = u"下班"
        pattern = re.compile(rule)
        match = pattern.search(self.exp_time)
        if match is not None:
            if self.tp.tunit[3] == -1: 
                self.tp.tunit[3] = RangeTimeEnum.night
                # 處理傾向於未來時間的情況
                self.preferFuture(3)
                self.isAllDayTime = False
                       
        rule = u"凌晨"
        pattern = re.compile(rule)
        match = pattern.search(self.exp_time)
        if match is not None:
            if self.tp.tunit[3] == -1: # 增加對沒有明確時間點，只寫了“凌晨”這種情況的處理
                self.tp.tunit[3] = RangeTimeEnum.day_break
                # 處理傾向於未來時間的情況
                self.preferFuture(3)
                self.isAllDayTime = False

        rule = u"早上|早晨|早間|晨間|今早|明早"
        pattern = re.compile(rule)
        match = pattern.search(self.exp_time)
        if match is not None:
            if self.tp.tunit[3] == -1:  # 增加對沒有明確時間點，只寫了“早上/早晨/早間”這種情況的處理
                self.tp.tunit[3] = RangeTimeEnum.early_morning
                # 處理傾向於未來時間的情況
                self.preferFuture(3)
                self.isAllDayTime = False

        rule = u"上午"
        pattern = re.compile(rule)
        match = pattern.search(self.exp_time)
        if match is not None:
            if self.tp.tunit[3] == -1:  # 增加對沒有明確時間點，只寫了“上午”這種情況的處理
                self.tp.tunit[3] = RangeTimeEnum.morning
                # 處理傾向於未來時間的情況
                self.preferFuture(3)
                self.isAllDayTime = False

        rule = u"(中午)|(午間)|(午休)"
        pattern = re.compile(rule)
        match = pattern.search(self.exp_time)
        if match is not None:
            if 0 <= self.tp.tunit[3] <= 10:
                self.tp.tunit[3] += 12
            if self.tp.tunit[3] == -1:  # 增加對沒有明確時間點，只寫了“中午/午間”這種情況的處理
                self.tp.tunit[3] = RangeTimeEnum.noon
            # 處理傾向於未來時間的情況
            self.preferFuture(3)
            self.isAllDayTime = False

        rule = u"(下午)|(午後)"
        pattern = re.compile(rule)
        match = pattern.search(self.exp_time)
        if match is not None:
            if 0 <= self.tp.tunit[3] <= 11:
                self.tp.tunit[3] += 12
            if self.tp.tunit[3] == -1:  # 增加對沒有明確時間點，只寫了“下午|午後”這種情況的處理
                self.tp.tunit[3] = RangeTimeEnum.afternoon
            # 處理傾向於未來時間的情況
            self.preferFuture(3)
            self.isAllDayTime = False
            
        rule = u"晚上|夜間|夜裡|今晚|明晚"
        pattern = re.compile(rule)
        match = pattern.search(self.exp_time)
        if match is not None:
            if 0 <= self.tp.tunit[3] <= 10:
                self.tp.tunit[3] += 12
            elif self.tp.tunit[3] == 12:
                self.tp.tunit[3] = 0
            elif self.tp.tunit[3] == -1:  # 增加對沒有明確時間點，只寫了“下午|午後”這種情況的處理
                self.tp.tunit[3] = RangeTimeEnum.lateNight
            # 處理傾向於未來時間的情況
            self.preferFuture(3)
            self.isAllDayTime = False

    def norm_setminute(self):
        """
        分-規範化方法：該方法識別時間表達式單元的分欄位
        :return:
        """
        rule = u"([0-9]+(?=分(?!鐘)))|((?<=((?<!小)[點時]))[0-5]?[0-9](?!刻))"
        pattern = re.compile(rule)
        match = pattern.search(self.exp_time)
        if match is not None:
            if match.group() != '':
                self.tp.tunit[4] = int(match.group())
                # 處理傾向於未來時間的情況
                # self.preferFuture(4)
                self.isAllDayTime = False
        # 加對一刻，半，3刻的正確識別（1刻為15分，半為30分，3刻為45分）
        rule = u"(?<=[點時])[1一]刻(?!鐘)"
        pattern = re.compile(rule)
        match = pattern.search(self.exp_time)
        if match is not None:
            self.tp.tunit[4] = 15
            # 處理傾向於未來時間的情況
            # self.preferFuture(4)
            self.isAllDayTime = False

        rule = u"(?<=[點時])半"
        pattern = re.compile(rule)
        match = pattern.search(self.exp_time)
        if match is not None:
            self.tp.tunit[4] = 30
            # 處理傾向於未來時間的情況
            self.preferFuture(4)
            self.isAllDayTime = False

        rule = u"(?<=[點時])[3三]刻(?!鐘)"
        pattern = re.compile(rule)
        match = pattern.search(self.exp_time)
        if match is not None:
            self.tp.tunit[4] = 45
            # 處理傾向於未來時間的情況
            # self.preferFuture(4)
            self.isAllDayTime = False

    def norm_setsecond(self):
        """
        添加了省略“秒”說法的時間：如17點15分32
        :return:
        """
        rule = u"([0-9]+(?=秒))|((?<=分)[0-5]?[0-9])"
        pattern = re.compile(rule)
        match = pattern.search(self.exp_time)
        if match is not None:
            self.tp.tunit[5] = int(match.group())
            self.isAllDayTime = False

    def norm_setSpecial(self):
        """
        特殊形式的規範化方法-該方法識別特殊形式的時間表達式單元的各個欄位
        :return:
        """
        rule = u"(?<!(周|週|星期|禮拜))([0-2]?[0-9]):[0-5]?[0-9]:[0-5]?[0-9]"
        pattern = re.compile(rule)
        match = pattern.search(self.exp_time)
        
        if match is not None:
            tmp_target = match.group()
            tmp_parser = tmp_target.split(":")
            self.tp.tunit[3] = int(tmp_parser[0])
            self.tp.tunit[4] = int(tmp_parser[1])
            self.tp.tunit[5] = int(tmp_parser[2])
            # 處理傾向於未來時間的情況
            self.preferFuture(3)
            self.isAllDayTime = False
        else:
            rule = u"(?<!(周|週|星期|禮拜))([0-2]?[0-9]):[0-5]?[0-9]"
            pattern = re.compile(rule)
            match = pattern.search(self.exp_time)
            if match is not None:
                tmp_target = match.group()
                tmp_parser = tmp_target.split(":")
                self.tp.tunit[3] = int(tmp_parser[0])
                self.tp.tunit[4] = int(tmp_parser[1])
                # 處理傾向於未來時間的情況
                self.preferFuture(3)
                self.isAllDayTime = False

        rule = u"[0-9]?[0-9]?[0-9]{2}-((10)|(11)|(12)|([1-9]))-((?<!\\d))([0-3][0-9]|[1-9])"
        pattern = re.compile(rule)
        match = pattern.search(self.exp_time)
        if match is not None:
            tmp_target = match.group()
            tmp_parser = tmp_target.split("-")
            self.tp.tunit[0] = int(tmp_parser[0])
            self.tp.tunit[1] = int(tmp_parser[1])
            self.tp.tunit[2] = int(tmp_parser[2])

        rule = u"((10)|(11)|(12)|([1-9]))/((?<!\\d))([0-3][0-9]|[1-9])/[0-9]?[0-9]?[0-9]{2}"
        pattern = re.compile(rule)
        match = pattern.search(self.exp_time)
        if match is not None:
            tmp_target = match.group()
            tmp_parser = tmp_target.split("/")
            self.tp.tunit[1] = int(tmp_parser[0])
            self.tp.tunit[2] = int(tmp_parser[1])
            self.tp.tunit[0] = int(tmp_parser[2])

        rule = u"[0-9]?[0-9]?[0-9]{2}\\.((10)|(11)|(12)|([1-9]))\\.((?<!\\d))([0-3][0-9]|[1-9])"
        pattern = re.compile(rule)
        match = pattern.search(self.exp_time)
        if match is not None:
            tmp_target = match.group()
            tmp_parser = tmp_target.split("\\.")
            self.tp.tunit[0] = int(tmp_parser[0])
            self.tp.tunit[1] = int(tmp_parser[1])
            self.tp.tunit[2] = int(tmp_parser[2])

    def norm_setBaseRelated(self):
        """
        設置以上文時間為基準的時間偏移計算
        :return:
        """
        cur = arrow.get(self.normalizer.timeBase, "YYYY-M-D-H-m-s")
        flag = [False, False, False, False]
#         print 'enter'
#         #-
#         rule = u"\\d+(?=(個)?小時[以之]?前)"
#         pattern = re.compile(rule)
#         match = pattern.search(self.exp_time)
#         if match is not None:
#             print 'QQ'
#             flag[3] = True
#             day = int(match.group())
#             cur = cur.shift(hours=-hour)

#         rule = u"\\d+(?=(個)?小時[以之]?後)"
#         pattern = re.compile(rule)
#         match = pattern.search(self.exp_time)
#         if match is not None:
#             print 'QQ'
#             flag[3] = True
#             day = int(match.group())
#             cur = cur.shift(hours=hour)
        #-
        
        rule = u"\\d+(?=天[以之]?前)"
        pattern = re.compile(rule)
        match = pattern.search(self.exp_time)
        if match is not None:
            flag[2] = True
            day = int(match.group())
            cur = cur.shift(days=-day)

        rule = u"\\d+(?=天[以之]?後)"
        pattern = re.compile(rule)
        match = pattern.search(self.exp_time)
        if match is not None:
            flag[2] = True
            day = int(match.group())
            cur = cur.shift(days=day)

        rule = u"\\d+(?=(個)?月[以之]?前)"
        pattern = re.compile(rule)
        match = pattern.search(self.exp_time)
        if match is not None:
            flag[1] = True
            month = int(match.group())
            cur = cur.shift(months=-month)

        rule = u"\\d+(?=(個)?月[以之]?後)"
        pattern = re.compile(rule)
        match = pattern.search(self.exp_time)
        if match is not None:
            flag[1] = True
            month = int(match.group())
            cur = cur.shift(months=month)

        rule = u"\\d+(?=年[以之]?前)"
        pattern = re.compile(rule)
        match = pattern.search(self.exp_time)
        if match is not None:
            flag[0] = True
            year = int(match.group())
            cur = cur.shift(years=-year)

        rule = u"\\d+(?=年[以之]?後)"
        pattern = re.compile(rule)
        match = pattern.search(self.exp_time)
        if match is not None:
            flag[0] = True
            year = int(match.group())
            cur = cur.shift(years=year)

        if flag[0] or flag[1] or flag[2] or flag[3]:
            self.tp.tunit[0] = int(cur.year)
        if flag[1] or flag[2] or flag[3]:
            self.tp.tunit[1] = int(cur.month)
        if flag[2] or flag[3]:
            self.tp.tunit[2] = int(cur.day)
        if flag[3]:
            self.tp.tunit[3] = int(cur.hour)

    # todo 時間長度相關
    def norm_setSpanRelated(self):
        """
        設置時間長度相關的時間表達式
        :return:
        """
        rule = u"\\d+(?=個月(?![以之]?[前後]))"
        pattern = re.compile(rule)
        match = pattern.search(self.exp_time)
        if match is not None:
            self.normalizer.isTimeSpan = True
            month = int(match.group())
            self.tp.tunit[1] = int(month)

        rule = u"\\d+(?=天(?![以之]?[前後]))"
        pattern = re.compile(rule)
        match = pattern.search(self.exp_time)
        if match is not None:
            self.normalizer.isTimeSpan = True
            day = int(match.group())
            self.tp.tunit[2] = int(day)

        rule = u"\\d+(?=(個)?小時(?![以之]?[前後]))"
        pattern = re.compile(rule)
        match = pattern.search(self.exp_time)
        if match is not None:
            self.normalizer.isTimeSpan = True
            hour = int(match.group())
            self.tp.tunit[3] = int(hour)

        rule = u"\\d+(?=分鐘(?![以之]?[前後]))"
        pattern = re.compile(rule)
        match = pattern.search(self.exp_time)
        if match is not None:
            self.normalizer.isTimeSpan = True
            minute = int(match.group())
            self.tp.tunit[4] = int(minute)

        rule = u"\\d+(?=秒鐘(?![以之]?[前後]))"
        pattern = re.compile(rule)
        match = pattern.search(self.exp_time)
        if match is not None:
            self.normalizer.isTimeSpan = True
            second = int(match.group())
            self.tp.tunit[5] = int(second)

        rule = u"\\d+(?=(個)?(周|週|星期|禮拜)(?![以之]?[前後]))"
        pattern = re.compile(rule)
        match = pattern.search(self.exp_time)
        
        if match is not None:
            self.normalizer.isTimeSpan = True
            week = int(match.group())
            if self.tp.tunit[2] == -1:
                self.tp.tunit[2] = 0
            self.tp.tunit[2] += int(week*7)
        

    def norm_setHoliday(self):
        
        ## 注意：包含數字的節日要加上阿拉伯數字文字
        rule = u"(清明)|(中元節)|(端午)|(勞動節)|(7夕)|(七夕)|(教師)|(聖誕)|(中秋)|(春節)|(元宵)|(國慶)|(雙十)|(雙10)|(元旦)|(重陽節)|(婦女節)"
        
        pattern = re.compile(rule)
        match = pattern.search(self.exp_time)
        if match is not None:
            if self.tp.tunit[0] == -1:
                self.tp.tunit[0] = int(self.normalizer.timeBase.split('-')[0])
            holi = match.group()
            if u'節' not in holi:
                holi += u'節'
            
#             print 'norm_setHoliday'
#             print self.normalizer.holi_solar
#             print type(self.normalizer.holi_lunar)
#             print holi
            holi = holi.encode('utf-8')

            if holi in self.normalizer.holi_solar:
#                 print 'norm_setHoliday > in holi_solar'
                date = self.normalizer.holi_solar[holi].split('-')
            if holi in self.normalizer.holi_lunar:
                date = self.normalizer.holi_lunar[holi].split('-')
#                 print 'norm_setHoliday > in holi_lunar'
                lsConverter = LunarSolarConverter()
                lunar = Lunar(self.tp.tunit[0], int(date[0]), int(date[1]), False)
                solar = lsConverter.LunarToSolar(lunar)
                self.tp.tunit[0] = solar.solarYear
                date[0] = solar.solarMonth
                date[1] = solar.solarDay
            self.tp.tunit[1] = int(date[0])
            self.tp.tunit[2] = int(date[1])


    def norm_setCurRelated(self):
        """
        設置當前時間相關的時間表達式
        :return:
        """
        cur = arrow.get(self.normalizer.timeBase, "YYYY-M-D-H-m-s")
        flag = [False, False, False]
        
        rule = u"前年"
        pattern = re.compile(rule)
        match = pattern.search(self.exp_time)
        if match is not None:
            flag[0] = True
            cur = cur.shift(years=-2)

        rule = u"去年"
        pattern = re.compile(rule)
        match = pattern.search(self.exp_time)
        if match is not None:
            flag[0] = True
            cur = cur.shift(years=-1)

        rule = u"今年"
        pattern = re.compile(rule)
        match = pattern.search(self.exp_time)
        if match is not None:
            flag[0] = True
            cur = cur.shift(years=0)

        rule = u"明年"
        pattern = re.compile(rule)
        match = pattern.search(self.exp_time)
        if match is not None:
            flag[0] = True
            cur = cur.shift(years=1)

        rule = u"隔年"
        pattern = re.compile(rule)
        match = pattern.search(self.exp_time)
        if match is not None:
            flag[0] = True
            cur = cur.shift(years=1)
            
        rule = u"後年"
        pattern = re.compile(rule)
        match = pattern.search(self.exp_time)
        if match is not None:
            flag[0] = True
            cur = cur.shift(years=2)

        rule = u"上(個)?月"
        pattern = re.compile(rule)
        match = pattern.search(self.exp_time)
        if match is not None:
            flag[1] = True
            cur = cur.shift(months=-1)

        rule = u"(本|這個)月"
        pattern = re.compile(rule)
        match = pattern.search(self.exp_time)
        if match is not None:
            flag[1] = True
            cur = cur.shift(months=0)

        rule = u"下(個)?月"
        pattern = re.compile(rule)
        match = pattern.search(self.exp_time)
        if match is not None:
            flag[1] = True
            cur = cur.shift(months=1)

        rule = u"大前天"
        pattern = re.compile(rule)
        match = pattern.search(self.exp_time)
        if match is not None:
            flag[2] = True
            cur = cur.shift(days=-3)

        rule = u"(?<!大)前天"
        pattern = re.compile(rule)
        match = pattern.search(self.exp_time)
        if match is not None:
            flag[2] = True
            cur = cur.shift(days=-2)

        rule = u"昨"
        pattern = re.compile(rule)
        match = pattern.search(self.exp_time)
        if match is not None:
            flag[2] = True
            cur = cur.shift(days=-1)

        rule = u"今(?!年)"
        pattern = re.compile(rule)
        match = pattern.search(self.exp_time)
        if match is not None:
            flag[2] = True
            cur = cur.shift(days=0)

        rule = u"明(?!年)"
        pattern = re.compile(rule)
        match = pattern.search(self.exp_time)
        if match is not None:
            flag[2] = True
            cur = cur.shift(days=1)

        rule = u"隔(?!年)"
        pattern = re.compile(rule)
        match = pattern.search(self.exp_time)
        if match is not None:
            flag[2] = True
            cur = cur.shift(days=1)
            
        rule = u"(?<!大)後天"
        pattern = re.compile(rule)
        match = pattern.search(self.exp_time)
        if match is not None:
            flag[2] = True
            cur = cur.shift(days=2)

        rule = u"大後天"
        pattern = re.compile(rule)
        match = pattern.search(self.exp_time)
        if match is not None:
            flag[2] = True
            cur = cur.shift(days=3)

        # todo 補充星期相關的預測 done
        rule = u"(?<=(上上(周|週|星期|禮拜)))[1-7]?"
        pattern = re.compile(rule)
        match = pattern.search(self.exp_time)
        
        if match is not None:
            flag[2] = True
            try:
                week = int(match.group())
            except:
                week = 1
            week -= 1
            span = week - cur.weekday()
            cur = cur.replace(weeks=-2, days=span)

        rule = u"(?<=((?<!上)上(周|週|星期|禮拜)))[1-7]?"
        pattern = re.compile(rule)
        match = pattern.search(self.exp_time)
       
        if match is not None:
            flag[2] = True
            try:
                week = int(match.group())
            except:
                week = 1
            week -= 1
            span = week - cur.weekday()
            cur = cur.replace(weeks=-1, days=span)

        rule = u"(?<=((?<!下)下(周|週|星期|禮拜)))[1-7]?"
        pattern = re.compile(rule)
        match = pattern.search(self.exp_time)

        if match is not None:
            flag[2] = True
            try:
                week = int(match.group())
            except:
                week = 1
            week -= 1
            span = week - cur.weekday()
            cur = cur.replace(weeks=1, days=span)

        rule = u"(?<=(下下(周|週|星期|禮拜)))[1-7]?"
        pattern = re.compile(rule)
        match = pattern.search(self.exp_time)
        
        if match is not None:
            flag[2] = True
            try:
                week = int(match.group())
            except:
                week = 1
            week -= 1
            span = week - cur.weekday()
            cur = cur.replace(weeks=2, days=span)

        rule = u"(?<=((?<!(上|下|個|[0-9]))(周|週|星期|禮拜)))[1-7]"
        pattern = re.compile(rule)
        match = pattern.search(self.exp_time)

        if match is not None:
            flag[2] = True
            try:
                week = int(match.group())
            except:
                week = 1
            week -= 1
            span = week - cur.weekday()
            cur = cur.replace(days=span)
            # 處理未來時間
            cur = self.preferFutureWeek(week, cur)

        if flag[0] or flag[1] or flag[2]:
            self.tp.tunit[0] = int(cur.year)
        if flag[1] or flag[2]:
            self.tp.tunit[1] = int(cur.month)
        if flag[2]:
            self.tp.tunit[2] = int(cur.day)

    def modifyTimeBase(self):
        """
        該方法用於更新timeBase使之具有上下文關聯性
        :return:
        """
        if not self.normalizer.isTimeSpan:
            if 30 <= self.tp.tunit[0] < 100:
                self.tp.tunit[0] = 1900 + self.tp.tunit[0]
            if 0 < self.tp.tunit[0] < 30:
                self.tp.tunit[0] = 2000 + self.tp.tunit[0]
            time_grid = self.normalizer.timeBase.split('-')
            arr = []
            for i in range(0, 6):
                if self.tp.tunit[i] == -1:
                    arr.append(str(time_grid[i]))
                else:
                    arr.append(str(self.tp.tunit[i]))
            self.normalizer.timeBase = '-'.join(arr)

    def preferFutureWeek(self, weekday, cur):
        # 1. 確認用戶選項
        if not self.normalizer.isPreferFuture:
            return cur
        # 2. 檢查被檢查的時間級別之前，是否沒有更高級的已經確定的時間，如果有，則不進行處理.
        for i in range(0, 2):
            if self.tp.tunit[i] != -1:
                return cur
        # 獲取當前是在周幾，如果識別到的時間小於當前時間，則識別時間為下一週
        tmp = arrow.get(self.normalizer.timeBase, "YYYY-M-D-H-m-s")
        curWeekday = tmp.weekday()
        if curWeekday > weekday:
            cur = cur.shift(days=7)
        return cur

    def preferFuture(self, checkTimeIndex):
        """
        如果用戶選項是傾向於未來時間，檢查checkTimeIndex所指的時間是否是過去的時間，如果是的話，將大一級的時間設為當前時間的+1。
        如在晚上說“早上8點看書”，則識別為明天早上;
        12月31日說“3號買菜”，則識別為明年1月的3號。
        :param checkTimeIndex: _tp.tunit時間數組的下標
        :return:
        """
        # 1. 檢查被檢查的時間級別之前，是否沒有更高級的已經確定的時間，如果有，則不進行處理.
        for i in range(0, checkTimeIndex):
            if self.tp.tunit[i] != -1:
                return
        # 2. 根據上下文補充時間
        self.checkContextTime(checkTimeIndex)
        # 3. 根據上下文補充時間後再次檢查被檢查的時間級別之前，是否沒有更高級的已經確定的時間，如果有，則不進行傾向處理.
        for i in range(0, checkTimeIndex):
            if self.tp.tunit[i] != -1:
                return
        # 4. 確認用戶選項
        if not self.normalizer.isPreferFuture:
            return
        # 5. 獲取當前時間，如果識別到的時間小於當前時間，則將其上的所有級別時間設置為當前時間，並且其上一級的時間步長+1
        time_arr = self.normalizer.timeBase.split('-')
        cur = arrow.get(self.normalizer.timeBase, "YYYY-M-D-H-m-s")
        cur_unit = int(time_arr[checkTimeIndex])
        if cur_unit < self.tp.tunit[checkTimeIndex]:
            return
        # 準備增加的時間單位是被檢查的時間的上一級，將上一級時間+1
        cur = self.addTime(cur, checkTimeIndex - 1)
        time_arr = cur.format("YYYY-M-D-H-m-s").split('-')
        for i in range(0, checkTimeIndex):
            self.tp.tunit[i] = int(time_arr[i])
            # if i == 1:
            #     self.tp.tunit[i] += 1

    def checkContextTime(self, checkTimeIndex):
        """
        根據上下文時間補充時間信息
        :param checkTimeIndex:
        :return:
        """
        for i in range(0, checkTimeIndex):
            if self.tp.tunit[i] == -1 and self.tp_origin.tunit[i] != -1:
                self.tp.tunit[i] = self.tp_origin.tunit[i]
        # 在處理小時這個級別時，如果上文時間是下午的且下文沒有主動聲明小時級別以上的時間，則也把下文時間設為下午
        if self.isFirstTimeSolveContext is True and checkTimeIndex == 3 and self.tp_origin.tunit[
            checkTimeIndex] >= 12 and self.tp.tunit[checkTimeIndex] < 12:
            self.tp.tunit[checkTimeIndex] += 12
        self.isFirstTimeSolveContext = False

    def addTime(self, cur, fore_unit):
        if fore_unit == 0:
            cur = cur.shift(years=1)
        elif fore_unit == 1:
            cur = cur.shift(months=1)
        elif fore_unit == 2:
            cur = cur.shift(days=1)
        elif fore_unit == 3:
            cur = cur.shift(hours=1)
        elif fore_unit == 4:
            cur = cur.shift(minutes=1)
        elif fore_unit == 5:
            cur = cur.shift(seconds=1)
        return cur


In [7]:

from pprint import pprint


class Lunar:
    def __init__(self, lunarYear, lunarMonth, lunarDay, isleap):
        self.isleap = isleap
        self.lunarDay = lunarDay
        self.lunarMonth = lunarMonth
        self.lunarYear = lunarYear


class Solar:
    def __init__(self, solarYear, solarMonth, solarDay):
        self.solarDay = solarDay
        self.solarMonth = solarMonth
        self.solarYear = solarYear


def GetBitInt(data, length, shift):
    return (data & (((1 << length) - 1) << shift)) >> shift


def SolarToInt(y, m, d):
    m = (m + 9) % 12
    y -= int(m / 10)
    return 365 * y + int(y / 4) - int(y / 100) + int(y / 400) + int((m * 306 + 5) / 10) + (d - 1)


def SolarFromInt(g):
    y = int((10000 * g + 14780) / 3652425)
    ddd = g - (365 * y + int(y / 4) - int(y / 100) + int(y / 400))
    if ddd < 0:
        y -= 1
        ddd = g - (365 * y + int(y / 4) - int(y / 100) + int(y / 400))

    mi = int((100 * ddd + 52) / 3060)
    mm = (mi + 2) % 12 + 1
    y += int((mi + 2) / 12)
    dd = ddd - int((mi * 306 + 5) / 10) + 1
    solar = Solar(y, mm, dd)
    return solar


class LunarSolarConverter:
    #####################################################################################
    # 1888~2111年農曆數據表
    # 農曆數據 每個元素的存儲格式如下：
    #   16~13    12          11~0
    #  閏幾月 閏月日數  1~12月份農曆日數(大小月)
    # 註：1、bit0表示農曆1月份日數，為1表示30天，為0表示29天。bit1表示農曆2月份日數，依次類推。
    #     2、bit12表示閏月日數，1為30天，0為29天。bit16~bit13表示第幾月是閏月(註：為0表示該年無閏月)
    # 數據來源參考: http://data.weather.gov.hk/gts/time/conversion1_text_c.htm
    #####################################################################################
    lunar_month_days = [1887, 0x1694, 0x16aa, 0x4ad5, 0xab6, 0xc4b7, 0x4ae, 0xa56, 0xb52a,
                        0x1d2a, 0xd54, 0x75aa, 0x156a, 0x1096d, 0x95c, 0x14ae, 0xaa4d, 0x1a4c, 0x1b2a, 0x8d55,
                        0xad4, 0x135a, 0x495d,
                        0x95c, 0xd49b, 0x149a, 0x1a4a, 0xbaa5, 0x16a8, 0x1ad4, 0x52da, 0x12b6, 0xe937, 0x92e,
                        0x1496, 0xb64b, 0xd4a,
                        0xda8, 0x95b5, 0x56c, 0x12ae, 0x492f, 0x92e, 0xcc96, 0x1a94, 0x1d4a, 0xada9, 0xb5a, 0x56c,
                        0x726e, 0x125c,
                        0xf92d, 0x192a, 0x1a94, 0xdb4a, 0x16aa, 0xad4, 0x955b, 0x4ba, 0x125a, 0x592b, 0x152a,
                        0xf695, 0xd94, 0x16aa,
                        0xaab5, 0x9b4, 0x14b6, 0x6a57, 0xa56, 0x1152a, 0x1d2a, 0xd54, 0xd5aa, 0x156a, 0x96c,
                        0x94ae, 0x14ae, 0xa4c,
                        0x7d26, 0x1b2a, 0xeb55, 0xad4, 0x12da, 0xa95d, 0x95a, 0x149a, 0x9a4d, 0x1a4a, 0x11aa5,
                        0x16a8, 0x16d4,
                        0xd2da, 0x12b6, 0x936, 0x9497, 0x1496, 0x1564b, 0xd4a, 0xda8, 0xd5b4, 0x156c, 0x12ae,
                        0xa92f, 0x92e, 0xc96,
                        0x6d4a, 0x1d4a, 0x10d65, 0xb58, 0x156c, 0xb26d, 0x125c, 0x192c, 0x9a95, 0x1a94, 0x1b4a,
                        0x4b55, 0xad4,
                        0xf55b, 0x4ba, 0x125a, 0xb92b, 0x152a, 0x1694, 0x96aa, 0x15aa, 0x12ab5, 0x974, 0x14b6,
                        0xca57, 0xa56, 0x1526,
                        0x8e95, 0xd54, 0x15aa, 0x49b5, 0x96c, 0xd4ae, 0x149c, 0x1a4c, 0xbd26, 0x1aa6, 0xb54,
                        0x6d6a, 0x12da, 0x1695d,
                        0x95a, 0x149a, 0xda4b, 0x1a4a, 0x1aa4, 0xbb54, 0x16b4, 0xada, 0x495b, 0x936, 0xf497,
                        0x1496, 0x154a, 0xb6a5,
                        0xda4, 0x15b4, 0x6ab6, 0x126e, 0x1092f, 0x92e, 0xc96, 0xcd4a, 0x1d4a, 0xd64, 0x956c,
                        0x155c, 0x125c, 0x792e,
                        0x192c, 0xfa95, 0x1a94, 0x1b4a, 0xab55, 0xad4, 0x14da, 0x8a5d, 0xa5a, 0x1152b, 0x152a,
                        0x1694, 0xd6aa,
                        0x15aa, 0xab4, 0x94ba, 0x14b6, 0xa56, 0x7527, 0xd26, 0xee53, 0xd54, 0x15aa, 0xa9b5, 0x96c,
                        0x14ae, 0x8a4e,
                        0x1a4c, 0x11d26, 0x1aa4, 0x1b54, 0xcd6a, 0xada, 0x95c, 0x949d, 0x149a, 0x1a2a, 0x5b25,
                        0x1aa4, 0xfb52,
                        0x16b4, 0xaba, 0xa95b, 0x936, 0x1496, 0x9a4b, 0x154a, 0x136a5, 0xda4, 0x15ac]
    # 額外添加數據，方便快速計算陰曆轉陽曆 每個元素的存儲格式如下：
    #    12~7         6~5    4~0
    #  離元旦多少天  春節月  春節日
    #####################################################################################
    solar_1_1 = [1887, 0xec04c, 0xec23f, 0xec435, 0xec649, 0xec83e, 0xeca51, 0xecc46, 0xece3a,
                 0xed04d, 0xed242, 0xed436, 0xed64a, 0xed83f, 0xeda53, 0xedc48, 0xede3d, 0xee050, 0xee244, 0xee439,
                 0xee64d,
                 0xee842, 0xeea36, 0xeec4a, 0xeee3e, 0xef052, 0xef246, 0xef43a, 0xef64e, 0xef843, 0xefa37, 0xefc4b,
                 0xefe41,
                 0xf0054, 0xf0248, 0xf043c, 0xf0650, 0xf0845, 0xf0a38, 0xf0c4d, 0xf0e42, 0xf1037, 0xf124a, 0xf143e,
                 0xf1651,
                 0xf1846, 0xf1a3a, 0xf1c4e, 0xf1e44, 0xf2038, 0xf224b, 0xf243f, 0xf2653, 0xf2848, 0xf2a3b, 0xf2c4f,
                 0xf2e45,
                 0xf3039, 0xf324d, 0xf3442, 0xf3636, 0xf384a, 0xf3a3d, 0xf3c51, 0xf3e46, 0xf403b, 0xf424e, 0xf4443,
                 0xf4638,
                 0xf484c, 0xf4a3f, 0xf4c52, 0xf4e48, 0xf503c, 0xf524f, 0xf5445, 0xf5639, 0xf584d, 0xf5a42, 0xf5c35,
                 0xf5e49,
                 0xf603e, 0xf6251, 0xf6446, 0xf663b, 0xf684f, 0xf6a43, 0xf6c37, 0xf6e4b, 0xf703f, 0xf7252, 0xf7447,
                 0xf763c,
                 0xf7850, 0xf7a45, 0xf7c39, 0xf7e4d, 0xf8042, 0xf8254, 0xf8449, 0xf863d, 0xf8851, 0xf8a46, 0xf8c3b,
                 0xf8e4f,
                 0xf9044, 0xf9237, 0xf944a, 0xf963f, 0xf9853, 0xf9a47, 0xf9c3c, 0xf9e50, 0xfa045, 0xfa238, 0xfa44c,
                 0xfa641,
                 0xfa836, 0xfaa49, 0xfac3d, 0xfae52, 0xfb047, 0xfb23a, 0xfb44e, 0xfb643, 0xfb837, 0xfba4a, 0xfbc3f,
                 0xfbe53,
                 0xfc048, 0xfc23c, 0xfc450, 0xfc645, 0xfc839, 0xfca4c, 0xfcc41, 0xfce36, 0xfd04a, 0xfd23d, 0xfd451,
                 0xfd646,
                 0xfd83a, 0xfda4d, 0xfdc43, 0xfde37, 0xfe04b, 0xfe23f, 0xfe453, 0xfe648, 0xfe83c, 0xfea4f, 0xfec44,
                 0xfee38,
                 0xff04c, 0xff241, 0xff436, 0xff64a, 0xff83e, 0xffa51, 0xffc46, 0xffe3a, 0x10004e, 0x100242,
                 0x100437,
                 0x10064b, 0x100841, 0x100a53, 0x100c48, 0x100e3c, 0x10104f, 0x101244, 0x101438, 0x10164c,
                 0x101842, 0x101a35,
                 0x101c49, 0x101e3d, 0x102051, 0x102245, 0x10243a, 0x10264e, 0x102843, 0x102a37, 0x102c4b,
                 0x102e3f, 0x103053,
                 0x103247, 0x10343b, 0x10364f, 0x103845, 0x103a38, 0x103c4c, 0x103e42, 0x104036, 0x104249,
                 0x10443d, 0x104651,
                 0x104846, 0x104a3a, 0x104c4e, 0x104e43, 0x105038, 0x10524a, 0x10543e, 0x105652, 0x105847,
                 0x105a3b, 0x105c4f,
                 0x105e45, 0x106039, 0x10624c, 0x106441, 0x106635, 0x106849, 0x106a3d, 0x106c51, 0x106e47,
                 0x10703c, 0x10724f,
                 0x107444, 0x107638, 0x10784c, 0x107a3f, 0x107c53, 0x107e48]

    def LunarToSolar(self, lunar):
        days = LunarSolarConverter.lunar_month_days[lunar.lunarYear - LunarSolarConverter.lunar_month_days[0]]
        leap = GetBitInt(days, 4, 13)
        offset = 0
        loopend = leap
        if not lunar.isleap:

            if lunar.lunarMonth <= leap or leap == 0:

                loopend = lunar.lunarMonth - 1

            else:

                loopend = lunar.lunarMonth

        for i in range(0, loopend):
            offset += GetBitInt(days, 1, 12 - i) == 1 and 30 or 29

        offset += lunar.lunarDay

        solar11 = LunarSolarConverter.solar_1_1[lunar.lunarYear - LunarSolarConverter.solar_1_1[0]]

        y = GetBitInt(solar11, 12, 9)
        m = GetBitInt(solar11, 4, 5)
        d = GetBitInt(solar11, 5, 0)

        return SolarFromInt(SolarToInt(y, m, d) + offset - 1)

    def SolarToLunar(self, solar):

        lunar = Lunar(0, 0, 0, False)
        index = solar.solarYear - LunarSolarConverter.solar_1_1[0]
        data = (solar.solarYear << 9) | (solar.solarMonth << 5) | solar.solarDay
        if LunarSolarConverter.solar_1_1[index] > data:
            index -= 1

        solar11 = LunarSolarConverter.solar_1_1[index]
        y = GetBitInt(solar11, 12, 9)
        m = GetBitInt(solar11, 4, 5)
        d = GetBitInt(solar11, 5, 0)
        offset = SolarToInt(solar.solarYear, solar.solarMonth, solar.solarDay) - SolarToInt(y, m, d)

        days = LunarSolarConverter.lunar_month_days[index]
        leap = GetBitInt(days, 4, 13)

        lunarY = index + LunarSolarConverter.solar_1_1[0]
        lunarM = 1
        offset += 1

        for i in range(0, 13):

            dm = GetBitInt(days, 1, 12 - i) == 1 and 30 or 29
            if offset > dm:

                lunarM += 1
                offset -= dm

            else:

                break

        lunarD = int(offset)
        lunar.lunarYear = lunarY
        lunar.lunarMonth = lunarM
        lunar.isleap = False
        if leap != 0 and lunarM > leap:

            lunar.lunarMonth = lunarM - 1
            if lunarM == leap + 1:
                lunar.isleap = True

        lunar.lunarDay = lunarD
        return lunar

    def __init__(self):
        pass


# if __name__ == '__main__':
#     converter = LunarSolarConverter()
#     solar = Solar(2111, 1, 25)
#     pprint(vars(solar))
#     lunar = converter.SolarToLunar(solar)
#     pprint(vars(lunar))
#     solar = converter.LunarToSolar(lunar)
#     pprint(vars(solar))
#     print(len(converter.solar_1_1))
#     print("Done")

In [8]:
import pickle
import os
import yaml

class TimeCalculateData:
    def __init__(self):
        self.interval = False
        self.repeat = False
        self.time_list = []
        self.time_interval_data_list = []
        
class TimeResultData:
    def __init__(self):
        self.interval = False
        self.repeat = False
#         self.time_list = []
        self.time_interval_data_list = []
        
class IntervalData:
    def __init__(self):
        self.start_date = ''
        self.start_time = ''
        self.end_date = ''
        self.end_time = ''

# 時間表達式識別的主要工作類
class TimeGenerator:
    
    def __init__(self, isPreferFuture=True, time_format = '%Y-%m-%d %H:%M:%S' ):
        self.isPreferFuture = isPreferFuture
        self.time_format = time_format
        self.pattern, self.holi_solar, self.holi_lunar = self.init()

#     def init(self):
#         fpath =  '/home/charles/nlp_resource/reg.pkl'
#         try:
#             with open(fpath, 'rb') as f:
#                 pattern = pickle.load(f)
#         except:
#             with open('/home/charles/nlp_resource/regex.txt', 'r') as f:
#                 content = f.read()
#             p = re.compile(content)
#             with open(fpath, 'wb') as f:
#                 pickle.dump(p, f)
#             with open(fpath, 'rb') as f:
#                 pattern = pickle.load(f)
#         with open(os.path.dirname(__file__) + '/home/charles/nlp_resource/holi_solar.json', 'r', encoding='utf-8') as f:
#             holi_solar = json.load(f)
#         with open(os.path.dirname(__file__) + '/home/charles/nlp_resource/holi_lunar.json', 'r', encoding='utf-8') as f:
#             holi_lunar = json.load(f)
#         return pattern, holi_solar, holi_lunar

    def init(self):
        regex = '((前|昨|今|明|隔|後)(天|日)?(早|晚)(晨|上|間)?)|(\d+個?[年月日天][以之]?[前後])|(\d+個?半?(小時|鐘頭|h|H))|(半個?(小時|鐘頭))|(\d+(分鐘|min))|([13]刻鐘)|((上|這|本|下)+(周|星期|禮拜|週)([一二三四五六七天日]|[1-7])?)|((周|星期|禮拜|週)([一二三四五六七天日]|[1-7]))|((早|晚)?([0-2]?[0-9](點|時)半)(am|AM|pm|PM)?)|((早|晚)?(\d+[:：]\d+([:：]\d+)*)\s*(am|AM|pm|PM)?)|((早|晚)?([0-2]?[0-9](點|時)[13一三]刻)(am|AM|pm|PM)?)|((早|晚)?(\d+[時點](\d+)?分?(\d+秒?)?)\s*(am|AM|pm|PM)?)|(大+(前|後)天)|(([零一二三四五六七八九十百千萬]+|\d+)世)|([0-9]?[0-9]?[0-9]{2}\.((10)|(11)|(12)|([1-9]))\.((?<!\\d))([0-3][0-9]|[1-9]))|(現在)|(屆時)|(這個月)|((數|多|多少|好幾|幾|差不多|近|前|後|上|左右)日)|(晚些時候)|(今年)|(長期)|(以前)|(過去)|(時期)|(時代)|(當時)|(近來)|(([零一二三四五六七八九十百千萬]+|\d+)夜)|(當前)|(日(數|多|多少|好幾|幾|差不多|近|前|後|上|左右))|((\d+)點)|(今年([零一二三四五六七八九十百千萬]+|\d+))|(\d+[:：]\d+(分|))|((\d+):(\d+))|(\d+/\d+/\d+)|(未來)|((充滿美麗、希望、挑戰的)?未來)|(最近)|(早上)|(早(數|多|多少|好幾|幾|差不多|近|前|後|上|左右))|(日前)|(新世紀)|(小時)|(([0-3][0-9]|[1-9])(日|號))|(明天|隔天)|(([0-3][0-9]|[1-9])[日號])|((數|多|多少|好幾|幾|差不多|近|前|後|上|左右)周|週)|((數|多|多少|好幾|幾|差不多|近|前|後|上|左右)([零一二三四五六七八九十百千萬]+|\d+)年)|([一二三四五六七八九十百千萬幾多]+[天日周週月年][後前左右]*)|(每[年月日天小時分秒鐘]+)|((\d+分)+(\d+秒)?)|([一二三四五六七八九十]+來?[歲年])|([新?|\d*]世紀末?)|((\d+)時)|(世紀)|(([零一二三四五六七八九十百千萬]+|\d+)歲)|(今年)|([星期周週禮拜]+[一二三四五六七])|(星期([零一二三四五六七八九十百千萬]+|\d+))|(([零一二三四五六七八九十百千萬]+|\d+)年)|([本後昨當新後明隔今去前那這][一二三四五六七八九十]?[年月日天])|(早|早晨|早上|上午|中午|午休|上班|下班|午後|下午|晚上|晚間|夜裡|夜|凌晨|深夜)|(回歸前後)|((\d+點)+(\d+分)?(\d+秒)?左右?)|((\d+)年代)|(本月(\d+))|(第(\d+)天)|((\d+)歲)|((\d+)年(\d+)月)|([去今明隔]?[年月](底|末))|(([零一二三四五六七八九十百千萬]+|\d+)世紀)|(昨天(數|多|多少|好幾|幾|差不多|近|前|後|上|左右)午)|(年度)|((數|多|多少|好幾|幾|差不多|近|前|後|上|左右)星期)|(年底)|([下個本]+賽季)|(今年(\d+)月(\d+)日)|((\d+)月(\d+)日(數|多|多少|好幾|幾|差不多|近|前|後|上|左右)午(\d+)時)|(今年晚些時候)|(兩個星期)|(過去(數|多|多少|好幾|幾|差不多|近|前|後|上|左右)周|週)|(本賽季)|(半個(數|多|多少|好幾|幾|差不多|近|前|後|上|左右))|(稍晚)|((\d+)號晚(數|多|多少|好幾|幾|差不多|近|前|後|上|左右))|(今(數|多|多少|好幾|幾|差不多|近|前|後|上|左右)(\d+)年)|(這個時候)|((數|多|多少|好幾|幾|差不多|近|前|後|上|左右)個小時)|(最(數|多|多少|好幾|幾|差不多|近|前|後|上|左右)(數|多|多少|好幾|幾|差不多|近|前|後|上|左右)年)|(凌晨)|((\d+)年(\d+)月(\d+)日)|((\d+)個月)|(今天早(數|多|多少|好幾|幾|差不多|近|前|後|上|左右))|(第[一二三四五六七八九十\d+]+季)|(當地時間)|(今(數|多|多少|好幾|幾|差不多|近|前|後|上|左右)([零一二三四五六七八九十百千萬]+|\d+)年)|(早晨)|(一段時間)|([本上]周|週[一二三四五六七])|(凌晨(\d+)點)|(去年(\d+)月(\d+)日)|(年關)|(如今)|((數|多|多少|好幾|幾|差不多|近|前|後|上|左右)小時)|(當晚)|((\d+)日晚(\d+)時)|(([零一二三四五六七八九十百千萬]+|\d+)(數|多|多少|好幾|幾|差不多|近|前|後|上|左右)午)|(每年(\d+)月(\d+)日)|(([零一二三四五六七八九十百千萬]+|\d+)周週)|((\d+)月)|(農曆)|(兩個小時)|(本(周|週)([零一二三四五六七八九十百千萬]+|\d+))|(長久)|(清晨)|((\d+)號晚)|(春節)|(星期日)|(聖誕)|((數|多|多少|好幾|幾|差不多|近|前|後|上|左右)段)|(現年)|(當日)|((數|多|多少|好幾|幾|差不多|近|前|後|上|左右)分鐘)|(\d+(天|日|周|週|月|年)(後|前|))|((文藝復興|巴洛克|前蘇聯|前一|暴力和專制|成年時期|古羅馬|我們所處的敏感)+時期)|((\d+)[年月天])|(清早)|(兩年)|((數|多|多少|好幾|幾|差不多|近|前|後|上|左右)午)|(昨天(數|多|多少|好幾|幾|差不多|近|前|後|上|左右)午(\d+)時)|(([零一二三四五六七八九十百千萬]+|\d+)(數|多|多少|好幾|幾|差不多|近|前|後|上|左右)年)|(今(數|多|多少|好幾|幾|差不多|近|前|後|上|左右)(\d+))|(聖誕節)|(學期)|(\d+來?分鐘)|(過去(數|多|多少|好幾|幾|差不多|近|前|後|上|左右)年)|(星期天)|(夜間)|((\d+)日凌晨)|(([零一二三四五六七八九十百千萬]+|\d+)月底)|(當天)|((\d+)日)|(((10)|(11)|(12)|([1-9]))月)|((數|多|多少|好幾|幾|差不多|近|前|後|上|左右)(數|多|多少|好幾|幾|差不多|近|前|後|上|左右)年)|(今年(\d+)月份)|(晚(數|多|多少|好幾|幾|差不多|近|前|後|上|左右)(\d+)時)|(連[年月日夜])|((\d+)年(\d+)月(\d+)日(數|多|多少|好幾|幾|差不多|近|前|後|上|左右)午)|((一|二|兩|三|四|五|六|七|八|九|十|百|千|萬|幾|多|上|\d+)+個?(天|日|周|週|月|年)(後|前|半|))|((勝利的)日子)|(青春期)|((數|多|多少|好幾|幾|差不多|近|前|後|上|左右)年)|(早(數|多|多少|好幾|幾|差不多|近|前|後|上|左右)([零一二三四五六七八九十百千萬]+|\d+)點(數|多|多少|好幾|幾|差不多|近|前|後|上|左右))|([0-9]{4}年)|(週末)|(([零一二三四五六七八九十百千萬]+|\d+)個(數|多|多少|好幾|幾|差不多|近|前|後|上|左右)小時)|(([(小學)|初中?|高中?|大學?|研][一二三四五六七八九十]?(\d+)?)?[上下]半?學期)|(([零一二三四五六七八九十百千萬]+|\d+)時期)|(午間)|(次年)|(這時候)|(農曆新年)|([春夏秋冬](天|季))|((\d+)天)|(元宵節)|((數|多|多少|好幾|幾|差不多|近|前|後|上|左右)分)|((\d+)月(\d+)日(數|多|多少|好幾|幾|差不多|近|前|後|上|左右)午)|(晚(數|多|多少|好幾|幾|差不多|近|前|後|上|左右)(\d+)時(\d+)分)|(傍晚)|(周|週([零一二三四五六七八九十百千萬]+|\d+))|((數|多|多少|好幾|幾|差不多|近|前|後|上|左右)午(\d+)時(\d+)分)|(同日)|((\d+)年(\d+)月底)|((\d+)分鐘)|((\d+)世紀)|(冬季)|(清明)(節)?|(青年節)|(教師節)|(中元節)|(端午)(節)?|(勞動節)|(7夕)(節)?|(建黨節)|(建軍節)|(中和節)|(聖誕)(節)?|(中秋)(節)?|(春節)|(元宵)(節)?|(航海日)|(兒童節)|(國慶)(節)?|(雙十)(節)?|雙10(節)?|(植樹節)|(元旦)|(重陽節)|(婦女節)|(記者節)|(年代)|(([零一二三四五六七八九十百千萬]+|\d+)年半)|(今年年底)|(新年)|(本週)|(當地時間星期([零一二三四五六七八九十百千萬]+|\d+))|(([零一二三四五六七八九十百千萬]+|\d+)(數|多|多少|好幾|幾|差不多|近|前|後|上|左右)歲)|(半小時)|(([零一二三四五六七八九十百千萬]+|\d+)週年)|((重要|最後)?時刻)|(([零一二三四五六七八九十百千萬]+|\d+)期間)|(週日)|(晚(數|多|多少|好幾|幾|差不多|近|前|後|上|左右))|(今後)|(([零一二三四五六七八九十百千萬]+|\d+)段時間)|(明年|隔年)|([12][09][0-9]{2}(年度?((前|昨|今|明|隔|後)(天|日)?(早|晚)(晨|上|間)?)|(\d+個?[年月日天小時分秒鐘][以之]?[前後])|(\d+個?半?(小時|鐘頭|h|H))|(半個?(小時|鐘頭))|(\d+(分鐘|min))|([13]刻鐘)|((上|這|本|下)+(周|星期|禮拜|週)([一二三四五六七天日]|[1-7])?)|((周|星期|禮拜|週)([一二三四五六七天日]|[1-7]))|((早|晚)?([0-2]?[0-9](點|時)半)(am|AM|pm|PM)?)|((早|晚)?(\d+[:：]\d+([:：]\d+)*)\s*(am|AM|pm|PM)?)|((早|晚)?([0-2]?[0-9](點|時)[13一三]刻)(am|AM|pm|PM)?)|((早|晚)?(\d+[時點](\d+)?分?(\d+秒?)?)\s*(am|AM|pm|PM)?)|(大+(前|後)天)|(([零一二三四五六七八九十百千萬]+|\d+)世)|([0-9]?[0-9]?[0-9]{2}\.((10)|(11)|(12)|([1-9]))\.((?<!\d))([0-3][0-9]|[1-9]))|(現在)|(屆時)|(這個月)|((數|多|多少|好幾|幾|差不多|近|前|後|上|左右)日)|(晚些時候)|(今年)|(長期)|(以前)|(過去)|(時期)|(時代)|(當時)|(近來)|(([零一二三四五六七八九十百千萬]+|\d+)夜)|(當前)|(日(數|多|多少|好幾|幾|差不多|近|前|後|上|左右))|((\d+)點)|(今年([零一二三四五六七八九十百千萬]+|\d+))|(\d+[:：]\d+(分|))|((\d+):(\d+))|(\d+/\d+/\d+)|(未來)|((充滿美麗、希望、挑戰的)?未來)|(最近)|(早上)|(早(數|多|多少|好幾|幾|差不多|近|前|後|上|左右))|(日前)|(新世紀)|(小時)|(([0-3][0-9]|[1-9])(日|號))|(明天|隔天)|(\d+)月|(([0-3][0-9]|[1-9])[日號])|((數|多|多少|好幾|幾|差不多|近|前|後|上|左右)周|週)|((數|多|多少|好幾|幾|差不多|近|前|後|上|左右)([零一二三四五六七八九十百千萬]+|\d+)年)|([一二三四五六七八九十百千萬幾多]+[天日周週月年][後前左右]*)|(每[年月日天小時分秒鐘]+)|((\d+分)+(\d+秒)?)|([一二三四五六七八九十]+來?[歲年])|([新?|\d*]世紀末?)|((\d+)時)|(世紀)|(([零一二三四五六七八九十百千萬]+|\d+)歲)|(今年)|([星期周禮拜週]+[一二三四五六七])|(星期([零一二三四五六七八九十百千萬]+|\d+))|(([零一二三四五六七八九十百千萬]+|\d+)年)|([本後昨當新後明隔今去前那這][一二三四五六七八九十]?[年月日天])|(早|早晨|早上|上午|中午|午休|上班|下班|午後|下午|晚上|晚間|夜裡|夜|凌晨|深夜)|(回歸前後)|((\d+點)+(\d+分)?(\d+秒)?左右?)|((\d+)年代)|(本月(\d+))|(第(\d+)天)|((\d+)歲)|((\d+)年(\d+)月)|([去今明隔]?[年月](底|末))|(([零一二三四五六七八九十百千萬]+|\d+)世紀)|(昨天(數|多|多少|好幾|幾|差不多|近|前|後|上|左右)午)|(年度)|((數|多|多少|好幾|幾|差不多|近|前|後|上|左右)星期)|(年底)|([下個本]+賽季)|(\d+)月(\d+)日|(\d+)月(\d+)|(今年(\d+)月(\d+)日)|((\d+)月(\d+)日(數|多|多少|好幾|幾|差不多|近|前|後|上|左右)午(\d+)時)|(今年晚些時候)|(兩個星期)|(過去(數|多|多少|好幾|幾|差不多|近|前|後|上|左右)周|週)|(本賽季)|(半個(數|多|多少|好幾|幾|差不多|近|前|後|上|左右))|(稍晚)|((\d+)號晚(數|多|多少|好幾|幾|差不多|近|前|後|上|左右))|(今(數|多|多少|好幾|幾|差不多|近|前|後|上|左右)(\d+)年)|(這個時候)|((數|多|多少|好幾|幾|差不多|近|前|後|上|左右)個小時)|(最(數|多|多少|好幾|幾|差不多|近|前|後|上|左右)(數|多|多少|好幾|幾|差不多|近|前|後|上|左右)年)|(凌晨)|((\d+)年(\d+)月(\d+)日)|((\d+)個月)|(今天早(數|多|多少|好幾|幾|差不多|近|前|後|上|左右))|(第[一二三四五六七八九十\d+]+季)|(當地時間)|(今(數|多|多少|好幾|幾|差不多|近|前|後|上|左右)([零一二三四五六七八九十百千萬]+|\d+)年)|(早晨)|(一段時間)|([本上]周|週[一二三四五六七])|(凌晨(\d+)點)|(去年(\d+)月(\d+)日)|(年關)|(如今)|((數|多|多少|好幾|幾|差不多|近|前|後|上|左右)小時)|(當晚)|((\d+)日晚(\d+)時)|(([零一二三四五六七八九十百千萬]+|\d+)(數|多|多少|好幾|幾|差不多|近|前|後|上|左右)午)|(每年(\d+)月(\d+)日)|(([零一二三四五六七八九十百千萬]+|\d+)周|週)|((\d+)月)|(農曆)|(兩個小時)|(本週([零一二三四五六七八九十百千萬]+|\d+))|(長久)|(清晨)|((\d+)號晚)|(春節)|(過年)|(星期日)|(聖誕)|((數|多|多少|好幾|幾|差不多|近|前|後|上|左右)段)|(現年)|(當日)|((數|多|多少|好幾|幾|差不多|近|前|後|上|左右)分鐘)|(\d+(天|日|周|週|月|年)(後|前|))|((文藝復興|巴洛克|前蘇聯|前一|暴力和專制|成年時期|古羅馬|我們所處的敏感)+時期)|((\d+)[年月天])|(清早)|(兩年)|((數|多|多少|好幾|幾|差不多|近|前|後|上|左右)午)|(昨天(數|多|多少|好幾|幾|差不多|近|前|後|上|左右)午(\d+)時)|(([零一二三四五六七八九十百千萬]+|\d+)(數|多|多少|好幾|幾|差不多|近|前|後|上|左右)年)|(今(數|多|多少|好幾|幾|差不多|近|前|後|上|左右)(\d+))|(聖誕節)|(學期)|(\d+來?分鐘)|(過去(數|多|多少|好幾|幾|差不多|近|前|後|上|左右)年)|(星期天)|(夜間)|((\d+)日凌晨)|(([零一二三四五六七八九十百千萬]+|\d+)月底)|(當天)|((\d+)日)|(((10)|(11)|(12)|([1-9]))月)|((數|多|多少|好幾|幾|差不多|近|前|後|上|左右)(數|多|多少|好幾|幾|差不多|近|前|後|上|左右)年)|(今年(\d+)月份)|(晚(數|多|多少|好幾|幾|差不多|近|前|後|上|左右)(\d+)時)|(連[年月日夜])|((\d+)年(\d+)月(\d+)日(數|多|多少|好幾|幾|差不多|近|前|後|上|左右)午)|((一|二|兩|三|四|五|六|七|八|九|十|百|千|萬|幾|多|上|\d+)+個?(天|日|周|週|月|年)(後|前|半|))|((勝利的)日子)|(青春期)|((數|多|多少|好幾|幾|差不多|近|前|後|上|左右)年)|(早(數|多|多少|好幾|幾|差不多|近|前|後|上|左右)([零一二三四五六七八九十百千萬]+|\d+)點(數|多|多少|好幾|幾|差不多|近|前|後|上|左右))|([0-9]{4}年)|(週末)|(([零一二三四五六七八九十百千萬]+|\d+)個(數|多|多少|好幾|幾|差不多|近|前|後|上|左右)小時)|(([(小學)|初中?|高中?|大學?|研][一二三四五六七八九十]?(\d+)?)?[上下]半?學期)|(([零一二三四五六七八九十百千萬]+|\d+)時期)|(午間)|(次年)|(這時候)|(農曆新年)|([春夏秋冬](天|季))|((\d+)天)|(元宵節)|((數|多|多少|好幾|幾|差不多|近|前|後|上|左右)分)|((\d+)月(\d+)日(數|多|多少|好幾|幾|差不多|近|前|後|上|左右)午)|(晚(數|多|多少|好幾|幾|差不多|近|前|後|上|左右)(\d+)時(\d+)分)|(傍晚)|(周([零一二三四五六七八九十百千萬]+|\d+))|((數|多|多少|好幾|幾|差不多|近|前|後|上|左右)午(\d+)時(\d+)分)|(同日)|((\d+)年(\d+)月底)|((\d+)分鐘)|((\d+)世紀)|(冬季)|(年代)|(([零一二三四五六七八九十百千萬]+|\d+)年半)|(今年年底)|(新年)|(本週)|(當地時間星期([零一二三四五六七八九十百千萬]+|\d+))|(([零一二三四五六七八九十百千萬]+|\d+)(數|多|多少|好幾|幾|差不多|近|前|後|上|左右)歲)|(半小時)|(每週)|(([零一二三四五六七八九十百千萬]+|\d+)週年)|((重要|最後)?時刻)|(([零一二三四五六七八九十百千萬]+|\d+)期間)|(週日)|(晚(數|多|多少|好幾|幾|差不多|近|前|後|上|左右))|(今後)|(([零一二三四五六七八九十百千萬]+|\d+)段時間)|(明年|隔年)|([12][09][0-9]{2}(年度?))|(([零一二三四五六七八九十百千萬]+|\d+)生)|(今天凌晨)|(過去(\d+)年)|(元月)|((\d+)月(\d+)日凌晨)|([前去今明隔後新]+年)|((\d+)月(\d+))|(夏天)|((\d+)日凌晨(\d+)時許)|((\d+)月(\d+)日)|((\d+)點半)|(去年底)|(最後一[天刻])|(最(數|多|多少|好幾|幾|差不多|近|前|後|上|左右)(數|多|多少|好幾|幾|差不多|近|前|後|上|左右)個月)|(聖誕節?)|(下?個?(星期|周|禮拜|週)(一|二|三|四|五|六|七|天))|((\d+)(數|多|多少|好幾|幾|差不多|近|前|後|上|左右)年)|(當天(數|多|多少|好幾|幾|差不多|近|前|後|上|左右)午)|(每年的(\d+)月(\d+)日)|((\d+)日晚(數|多|多少|好幾|幾|差不多|近|前|後|上|左右))|(星期([零一二三四五六七八九十百千萬]+|\d+)晚)|(深夜)|(現如今)|([上中下]+午)|(第(一|二|三|四|五|六|七|八|九|十|百|千|萬|幾|多|\d+)+個?(天|日|周|週|月|年))|(昨晚)|(近年)|(今天清晨)|(中旬)|(星期([零一二三四五六七八九十百千萬]+|\d+)早)|(([零一二三四五六七八九十百千萬]+|\d+)戰期間)|(星期)|(昨天晚(數|多|多少|好幾|幾|差不多|近|前|後|上|左右))|(較早時)|(個(數|多|多少|好幾|幾|差不多|近|前|後|上|左右)小時)|((民主高中|我們所處的|復仇主義和其它危害人類的災難性疾病盛行的|快速承包電影主權的|恢復自我美德|人類審美力基礎設施|飽受暴力、野蠻、流血、仇恨、嫉妒的|童年|艱苦的童年)+時代)|(元旦)|(([零一二三四五六七八九十百千萬]+|\d+)個禮拜)|(昨日)|([年月]初)|((\d+)年的(\d+)月)|(每年)|(([零一二三四五六七八九十百千萬]+|\d+)月份)|(今年(\d+)月(\d+)號)|(今年([零一二三四五六七八九十百千萬]+|\d+)月)|((\d+)月底)|(未來(\d+)年)|(第([零一二三四五六七八九十百千萬]+|\d+)季)|(\d?多年)|(([零一二三四五六七八九十百千萬]+|\d+)個星期)|((\d+)年([零一二三四五六七八九十百千萬]+|\d+)月)|([下上中]午)|(早(數|多|多少|好幾|幾|差不多|近|前|後|上|左右)(\d+)點)|((數|多|多少|好幾|幾|差不多|近|前|後|上|左右)月)|(([零一二三四五六七八九十百千萬]+|\d+)個(數|多|多少|好幾|幾|差不多|近|前|後|上|左右)月)|(同([零一二三四五六七八九十百千萬]+|\d+)天)|((\d+)號凌晨)|(夜裡)|(兩個(數|多|多少|好幾|幾|差不多|近|前|後|上|左右)小時)|(昨天)|(羅馬時代)|(目(數|多|多少|好幾|幾|差不多|近|前|後|上|左右))|(([零一二三四五六七八九十百千萬]+|\d+)月)|((\d+)年(\d+)月(\d+)號)|(((10)|(11)|(12)|([1-9]))月份?)|([12][0-9]世紀)|((數|多|多少|好幾|幾|差不多|近|前|後|上|左右)([零一二三四五六七八九十百千萬]+|\d+)天)|(工作日)|(稍後)|((\d+)號(數|多|多少|好幾|幾|差不多|近|前|後|上|左右)午)|(未來([零一二三四五六七八九十百千萬]+|\d+)年)|([0-9]+[天日周週月年][後前左右]*)|(([零一二三四五六七八九十百千萬]+|\d+)日(數|多|多少|好幾|幾|差不多|近|前|後|上|左右)午)|(最(數|多|多少|好幾|幾|差不多|近|前|後|上|左右)([零一二三四五六七八九十百千萬]+|\d+)刻)|(很久)|((\d+)(數|多|多少|好幾|幾|差不多|近|前|後|上|左右)歲)|(去年(\d+)月(\d+)號)|(兩個月)|((數|多|多少|好幾|幾|差不多|近|前|後|上|左右)午(\d+)時)|(古代)|(兩天)|(\d+個?(小時|星期))|((\d+)年半)|(較早)|(([零一二三四五六七八九十百千萬]+|\d+)個小時)|([一二三四五六七八九十]+週年)|(星期([零一二三四五六七八九十百千萬]+|\d+)(數|多|多少|好幾|幾|差不多|近|前|後|上|左右)午)|(時刻)|((\d+天)+(\d+點)?(\d+分)?(\d+秒)?)|((\d+)日([零一二三四五六七八九十百千萬]+|\d+)時)|((\d+)週年)|(([零一二三四五六七八九十百千萬]+|\d+)早)|(([零一二三四五六七八九十百千萬]+|\d+)日)|(去年(\d+)月)|(過去([零一二三四五六七八九十百千萬]+|\d+)年)|((\d+)個星期)|((數|多|多少|好幾|幾|差不多|近|前|後|上|左右)(數|多|多少|好幾|幾|差不多|近|前|後|上|左右)天)|(執政期間)|([當前昨今明隔後春夏秋冬]+天)|(去年(\d+)月份)|(今(數|多|多少|好幾|幾|差不多|近|前|後|上|左右))|((\d+)周)|(兩星期)|(([零一二三四五六七八九十百千萬]+|\d+)年代)|((數|多|多少|好幾|幾|差不多|近|前|後|上|左右)天)|(昔日)|(兩個半月)|([印尼|北京|美國]?當地時間)|(連日)|(本月(\d+)日)|(第([零一二三四五六七八九十百千萬]+|\d+)天)|((\d+)點(\d+)分)|([長近多]年)|((\d+)日(數|多|多少|好幾|幾|差不多|近|前|後|上|左右)午(\d+)時)|(那時)|(冷戰時代)|(([零一二三四五六七八九十百千萬]+|\d+)天)|(這個星期)|(去年)|(昨天傍晚)|(近期)|(星期([零一二三四五六七八九十百千萬]+|\d+)早些時候)|((\d+)([零一二三四五六七八九十百千萬]+|\d+)年)|((數|多|多少|好幾|幾|差不多|近|前|後|上|左右)兩個月)|((\d+)個小時)|(([零一二三四五六七八九十百千萬]+|\d+)個月)|(當年)|(本月)|((數|多|多少|好幾|幾|差不多|近|前|後|上|左右)([零一二三四五六七八九十百千萬]+|\d+)個月)|((\d+)點(數|多|多少|好幾|幾|差不多|近|前|後|上|左右))|(目前)|(去年([零一二三四五六七八九十百千萬]+|\d+)月)|((\d+)時(\d+)分)|(每月)|((數|多|多少|好幾|幾|差不多|近|前|後|上|左右)段時間)|((\d+)日晚)|(早(數|多|多少|好幾|幾|差不多|近|前|後|上|左右)(\d+)點(數|多|多少|好幾|幾|差不多|近|前|後|上|左右))|(下旬)|((\d+)月份)|(逐年)|(稍(數|多|多少|好幾|幾|差不多|近|前|後|上|左右))|((\d+)年)|(月底)|(這個月)|((\d+)年(\d+)個月)|(\d+大壽)|(周([零一二三四五六七八九十百千萬]+|\d+)早(數|多|多少|好幾|幾|差不多|近|前|後|上|左右))|(半年)|(今日)|(末日)|(昨天深夜)|(今年(\d+)月)|((\d+)月(\d+)號)|((\d+)日夜)|((早些|某個|晚間|本星期早些|前些)+時候)|(同年)|((北京|那個|更長的|最終衝突的)時間)|(每個月)|(一早)|((\d+)來?[歲年])|((數|多|多少|好幾|幾|差不多|近|前|後|上|左右)個月)|([鼠牛虎兔龍蛇馬羊猴雞狗豬]年)|(季度)|(早些時候)|(今天)|(每天)|(年半)|(下(個)?月)|(午後)|((\d+)日(數|多|多少|好幾|幾|差不多|近|前|後|上|左右)午)|((數|多|多少|好幾|幾|差不多|近|前|後|上|左右)個星期)|(今天(數|多|多少|好幾|幾|差不多|近|前|後|上|左右)午)|(同[一二三四五六七八九十][年|月|天])|(T\d+:\d+:\d+)|(\d+/\d+/\d+:\d+:\d+.\d+)|(\?\?\?\?-\?\?-\?\?T\d+:\d+:\d+)|(\d+-\d+-\d+T\d+:\d+:\d+)|(\d+/\d+/\d+ \d+:\d+:\d+.\d+)|(\d+-\d+-\d+|[0-9]{8})|(((\d+)年)?((10)|(11)|(12)|([1-9]))月(\d+))|((\d[\.\-])?((10)|(11)|(12)|([1-9]))[\.\-](\d+))))|(([零一二三四五六七八九十百千萬]+|\d+)生)|(今天凌晨)|(過去(\d+)年)|(元月)|((\d+)月(\d+)日凌晨)|([前去今明隔後新]+年)|((\d+)月(\d+))|(夏天)|((\d+)日凌晨(\d+)時許)|((\d+)月(\d+)日)|((\d+)點半)|(去年底)|(最後一[天刻])|(最(數|多|多少|好幾|幾|差不多|近|前|後|上|左右)(數|多|多少|好幾|幾|差不多|近|前|後|上|左右)個月)|(聖誕節?)|(下?個?(星期|周)(一|二|三|四|五|六|七|天))|((\d+)(數|多|多少|好幾|幾|差不多|近|前|後|上|左右)年)|(當天(數|多|多少|好幾|幾|差不多|近|前|後|上|左右)午)|(每年的(\d+)月(\d+)日)|((\d+)日晚(數|多|多少|好幾|幾|差不多|近|前|後|上|左右))|(星期([零一二三四五六七八九十百千萬]+|\d+)晚)|(深夜)|(現如今)|([上中下]+午)|(第(一|二|三|四|五|六|七|八|九|十|百|千|萬|幾|多|\d+)+個?(天|日|周|週|月|年))|(昨晚)|(近年)|(今天清晨)|(中旬)|(星期([零一二三四五六七八九十百千萬]+|\d+)早)|(([零一二三四五六七八九十百千萬]+|\d+)戰期間)|(星期)|(昨天晚(數|多|多少|好幾|幾|差不多|近|前|後|上|左右))|(較早時)|(個(數|多|多少|好幾|幾|差不多|近|前|後|上|左右)小時)|((民主高中|我們所處的|復仇主義和其它危害人類的災難性疾病盛行的|快速承包電影主權的|恢復自我美德|人類審美力基礎設施|飽受暴力、野蠻、流血、仇恨、嫉妒的|童年|艱苦的童年)+時代)|(元旦)|(([零一二三四五六七八九十百千萬]+|\d+)個禮拜)|(昨日)|([年月]初)|((\d+)年的(\d+)月)|(每年)|(([零一二三四五六七八九十百千萬]+|\d+)月份)|(今年(\d+)月(\d+)號)|(今年([零一二三四五六七八九十百千萬]+|\d+)月)|((\d+)月底)|(未來(\d+)年)|(第([零一二三四五六七八九十百千萬]+|\d+)季)|(\d?多年)|(([零一二三四五六七八九十百千萬]+|\d+)個星期)|((\d+)年([零一二三四五六七八九十百千萬]+|\d+)月)|([下上中]午)|(早(數|多|多少|好幾|幾|差不多|近|前|後|上|左右)(\d+)點)|((數|多|多少|好幾|幾|差不多|近|前|後|上|左右)月)|(([零一二三四五六七八九十百千萬]+|\d+)個(數|多|多少|好幾|幾|差不多|近|前|後|上|左右)月)|(同([零一二三四五六七八九十百千萬]+|\d+)天)|((\d+)號凌晨)|(夜裡)|(兩個(數|多|多少|好幾|幾|差不多|近|前|後|上|左右)小時)|(昨天)|(羅馬時代)|(目(數|多|多少|好幾|幾|差不多|近|前|後|上|左右))|(([零一二三四五六七八九十百千萬]+|\d+)月)|((\d+)年(\d+)月(\d+)號)|(((10)|(11)|(12)|([1-9]))月份?)|([12][0-9]世紀)|((數|多|多少|好幾|幾|差不多|近|前|後|上|左右)([零一二三四五六七八九十百千萬]+|\d+)天)|(工作日)|(稍後)|((\d+)號(數|多|多少|好幾|幾|差不多|近|前|後|上|左右)午)|(未來([零一二三四五六七八九十百千萬]+|\d+)年)|([0-9]+[天日周週月年][後前左右]*)|(([零一二三四五六七八九十百千萬]+|\d+)日(數|多|多少|好幾|幾|差不多|近|前|後|上|左右)午)|(最(數|多|多少|好幾|幾|差不多|近|前|後|上|左右)([零一二三四五六七八九十百千萬]+|\d+)刻)|(很久)|((\d+)(數|多|多少|好幾|幾|差不多|近|前|後|上|左右)歲)|(去年(\d+)月(\d+)號)|(兩個月)|((數|多|多少|好幾|幾|差不多|近|前|後|上|左右)午(\d+)時)|(古代)|(兩天)|(\d+個?(小時|星期))|((\d+)年半)|(較早)|(([零一二三四五六七八九十百千萬]+|\d+)個小時)|([一二三四五六七八九十]+週年)|(星期([零一二三四五六七八九十百千萬]+|\d+)(數|多|多少|好幾|幾|差不多|近|前|後|上|左右)午)|(時刻)|((\d+天)+(\d+點)?(\d+分)?(\d+秒)?)|((\d+)日([零一二三四五六七八九十百千萬]+|\d+)時)|((\d+)週年)|(([零一二三四五六七八九十百千萬]+|\d+)早)|(([零一二三四五六七八九十百千萬]+|\d+)日)|(去年(\d+)月)|(過去([零一二三四五六七八九十百千萬]+|\d+)年)|((\d+)個星期)|((數|多|多少|好幾|幾|差不多|近|前|後|上|左右)(數|多|多少|好幾|幾|差不多|近|前|後|上|左右)天)|(執政期間)|([當前昨今明隔後春夏秋冬]+天)|(去年(\d+)月份)|(今(數|多|多少|好幾|幾|差不多|近|前|後|上|左右))|((\d+)周)|(兩星期)|(([零一二三四五六七八九十百千萬]+|\d+)年代)|((數|多|多少|好幾|幾|差不多|近|前|後|上|左右)天)|(昔日)|(兩個半月)|([印尼|北京|美國]?當地時間)|(連日)|(本月(\d+)日)|(第([零一二三四五六七八九十百千萬]+|\d+)天)|((\d+)點(\d+)分)|([長近多]年)|((\d+)日(數|多|多少|好幾|幾|差不多|近|前|後|上|左右)午(\d+)時)|(那時)|(冷戰時代)|(([零一二三四五六七八九十百千萬]+|\d+)天)|(這個星期)|(去年)|(昨天傍晚)|(近期)|(星期([零一二三四五六七八九十百千萬]+|\d+)早些時候)|((\d+)([零一二三四五六七八九十百千萬]+|\d+)年)|((數|多|多少|好幾|幾|差不多|近|前|後|上|左右)兩個月)|((\d+)個小時)|(([零一二三四五六七八九十百千萬]+|\d+)個月)|(當年)|(本月)|((數|多|多少|好幾|幾|差不多|近|前|後|上|左右)([零一二三四五六七八九十百千萬]+|\d+)個月)|((\d+)點(數|多|多少|好幾|幾|差不多|近|前|後|上|左右))|(目前)|(去年([零一二三四五六七八九十百千萬]+|\d+)月)|((\d+)時(\d+)分)|(每月)|((數|多|多少|好幾|幾|差不多|近|前|後|上|左右)段時間)|((\d+)日晚)|(早(數|多|多少|好幾|幾|差不多|近|前|後|上|左右)(\d+)點(數|多|多少|好幾|幾|差不多|近|前|後|上|左右))|(下旬)|((\d+)月份)|(逐年)|(稍(數|多|多少|好幾|幾|差不多|近|前|後|上|左右))|((\d+)年)|(月底)|(這個月)|((\d+)年(\d+)個月)|(\d+大壽)|(周|週([零一二三四五六七八九十百千萬]+|\d+)早(數|多|多少|好幾|幾|差不多|近|前|後|上|左右))|(半年)|(今日)|(末日)|(昨天深夜)|(今年(\d+)月)|((\d+)月(\d+)號)|((\d+)日夜)|((早些|某個|晚間|本星期早些|前些)+時候)|(同年)|((北京|那個|更長的|最終衝突的)時間)|(每個月)|(一早)|((\d+)來?[歲年])|((數|多|多少|好幾|幾|差不多|近|前|後|上|左右)個月)|([鼠牛虎兔龍蛇馬羊猴雞狗豬]年)|(季度)|(早些時候)|(今天)|(每天)|(年半)|(下(個)?月)|(午後)|((\d+)日(數|多|多少|好幾|幾|差不多|近|前|後|上|左右)午)|((數|多|多少|好幾|幾|差不多|近|前|後|上|左右)個星期)|(\d+秒)|(今天(數|多|多少|好幾|幾|差不多|近|前|後|上|左右)午)|(同[一二三四五六七八九十][年|月|天])|(T\d+:\d+:\d+)|(\d+/\d+/\d+:\d+:\d+.\d+)|(\?\?\?\?-\?\?-\?\?T\d+:\d+:\d+)|(\d+-\d+-\d+T\d+:\d+:\d+)|(\d+/\d+/\d+ \d+:\d+:\d+.\d+)|(\d+-\d+-\d+|[0-9]{8})|(((\d+)年)?((10)|(11)|(12)|([1-9]))月(\d+))|((\d[\.\-])?((10)|(11)|(12)|([1-9]))[\.\-](\d+))'

        holi_solar = {'聖誕節':'12-25', '元旦節': '01-01','國慶節': '10-10','勞動節': '05-01', '婦女節':'03-08', '清明節':'04-05','教師節':'09-28','雙十節': '10-10','雙10節': '10-10'}
        holi_lunar = {'中秋節': '08-15', '中元節': '07-15', '端午節': '05-05', '春節': '01-01','元宵節': '01-15',  '重陽節': '09-09', '7夕節':'07-07', '七夕節':'07-07'}
        return re.compile(regex.decode('utf-8')), holi_solar, holi_lunar
    
    ##  時間處理
    def get_time(self, target):
        """
        判斷時間對外接口
        :param timeBase: 基準時間點
        :param target: 待分析字元串
        :return: 時間單元list
        """
        timeBase=arrow.now()

#         t_base = timeBase.strftime(self.time_format)
#         target=target.decode('utf-8')
        res = self.parse(target, timeBase)
        time_dict = yaml.safe_load(res)
        #     print time_dict
        has_time = time_dict.get('timestamp', None)
        if has_time == None: 
            return ''

        time_list = time_dict['timestamp']
            
        td = TimeCalculateData()

#         td.target = target.encode('utf8')
#         print  td.target
        td.time_list = time_list
        td.time_interval_data_list = []
        
        ## 判斷是否區間
        td = self.__interval_check(target,  td)
        
        ## 判斷是否重複
        td = self.__repeat_check(target,  td)
        
        ##組成回傳結果
        td = self.__time_result_parser(td)
#         ## TODO: datetime output 
        
        ## 回傳格式
        tr = TimeResultData()
        tr.repeat = td.repeat
        tr.interval = td.interval
        tr.time_interval_data_list = td.time_interval_data_list
        return tr
    

    def parse(self, target, timeBase=arrow.now()):
        """
        TimeGenerator的構造方法，timeBase取默認的系統當前時間
        :param timeBase: 基準時間點
        :param target: 待分析字元串
        :return: 時間單元數組
        """
        self.isTimeSpan = False
        self.invalidSpan = False
        self.timeSpan = ''
        self.target = target
        self.timeBase = arrow.get(timeBase).format('YYYY-M-D-H-m-s')
        self.oldTimeBase = self.timeBase
        self.__preHandling()
        self.timeToken = self.__timeEx()
        dic = {}
        res = self.timeToken
        
#         print 'TimeGenerator > parse > self.isTimeSpan = ' + str(self.isTimeSpan)

#         if self.isTimeSpan:
#             if self.invalidSpan:
#                 dic['error'] = 'no time pattern could be extracted.'
#             else:
#                 dic['type'] = 'timedelta'
#                 dic['timedelta'] = self.timeSpan
#         else:

#             print 'TimeGenerator > parse > res = ' + str(res)

        timestamp_list = []
        if len(res) == 0:
            dic['error'] = 'no time pattern could be extracted.'
        elif len(res) == 1:
            dic['type'] = 'timestamp'
            for r in res:
                timestamp_list.append(r.time.format("YYYY-MM-DD HH:mm:ss"))
            dic['timestamp'] = timestamp_list
#             dic['timestamp'] = res[0].time.format("YYYY-MM-DD HH:mm:ss")
        else:
            dic['type'] = 'timestamp'
        
            for r in res:
                timestamp_list.append(r.time.format("YYYY-MM-DD HH:mm:ss"))
            dic['timestamp'] = timestamp_list
#             dic['type'] = 'timespan'
#             dic['timespan'] = [res[0].time.format("YYYY-MM-DD HH:mm:ss"), res[1].time.format("YYYY-MM-DD HH:mm:ss")]
        return json.dumps(dic)

    def __preHandling(self):
        """
        待匹配字元串的清理空白符和語氣助詞以及大寫數字轉化的預處理
        :return:
        """
#         print '__preHandling  = ' + self.target
        self.target = sp.delKeyword(self.target, u"\\s+")  # 清理空白符
#         print '__preHandling > delKeyword empty = ' + self.target
        self.target = sp.delKeyword(self.target, u"[的]+")  # 清理語氣助詞
#         print '__preHandling > delKeyword D = ' + self.target
        self.target = sp.numberTranslator(self.target)  # 大寫數字轉化
#         print '__preHandling > delKeyword numberTranslator= ' + self.target


    def __timeEx(self):
        """
        :param target: 輸入文本字元串
        :param timeBase: 輸入基準時間
        :return: TimeUnit[]時間表達式類型數組
        """
        startline = -1
        endline = -1
        rpointer = 0
        temp = []
        
#         print '__timeEx :  self.target = ' + self.target
        
        match = self.pattern.finditer(self.target)
        for m in match:
            startline = m.start()
            if startline == endline:
                rpointer -= 1
                temp[rpointer] = temp[rpointer] + m.group()
            else:
                temp.append(m.group())
            endline = m.end()
            rpointer += 1
            
        res = []
        # 時間上下文： 前一個識別出來的時間會是下一個時間的上下文，用於處理：週六3點到5點這樣的多個時間的識別，第二個5點應識別到是週六的。
        contextTp = TimePoint()
        
#         print '__timeEx :  rpointer = ' + str(rpointer)
        for i in range(0, rpointer):
            # 這裡是一個類嵌套了一個類
            res.append(TimeUnit(temp[i], self, contextTp))
            contextTp = res[i].tp
        res = self.__filterTimeUnit(res)

        return res

    def __filterTimeUnit(self, tu_arr):
        """
        過濾timeUnit中無用的識別詞。無用識別詞識別出的時間是1970.01.01 00:00:00(fastTime=0)
        :param tu_arr:
        :return:
        """
        if (tu_arr is None) or (len(tu_arr) < 1):
            return tu_arr
        res = []
        for tu in tu_arr:
            if tu.time.timestamp != 0:
                res.append(tu)
        return res
    
    def __time_result_parser(self, time_data):  
        
#         if time_data.time_type == 'interval':
        if time_data.interval == True:
            
            interval_data = IntervalData()
            
            index = 0
            for t in time_data.time_list:

                t_date_time_list = t.split(' ')

                if index == 0:
                    interval_data.start_date = t_date_time_list[0]
                    interval_data.start_time = t_date_time_list[1]

                if index == (len(time_data.time_list) - 1):
                    interval_data.end_date = t_date_time_list[0]
                    interval_data.end_time = t_date_time_list[1]  
                
                index =  index + 1
            
            time_data.time_interval_data_list.append(interval_data)
            return time_data
        
        else:

            for t in time_data.time_list:

                interval_data = IntervalData()

                t_date_time_list = t.split(' ')

                interval_data.start_date = t_date_time_list[0]
                interval_data.start_time = t_date_time_list[1]
                time_data.time_interval_data_list.append(interval_data)

            return time_data
            
    
    def __repeat_check(self, target, time_data):
        
#             rule = u"(每個|每次|重複|循環|每年|每月|每日|每天|每周|每週)"
#             rule = u"(每[個次年月日天周週]|重複|循環)"
            rule = u"(每|重複|循環)"
            pattern = re.compile(rule)
            match = pattern.search(target)
            
            if match is not None:
#                 time_data.time_type = 'repeat'
                time_data.repeat = True
                return time_data
            
            return time_data
    
    def __interval_check(self, target, time_data):
        """
        處理年月日時的詳細時間區間 EX: 20181日到5日 >> 20180301 ~ 20180305 or 一周 >> 列出一個禮拜
        :param target: 輸入文本字元串
        :param time: 時間區間
        :return:
        """

        if  len(time_data.time_list) == 0:
            return time_data
        
        else: 

            ## 本週一～日
            (check, time_data) = self.__this_week_interval_check(target, time_data)
            if check:
                time_data.interval =True
                return time_data
            
            ## 判斷未來or過去一周時間
            (check, time_data) = self.__past_interval_check(target, time_data)
            if check:
                time_data.interval =True
                return time_data
            
            ## 判斷區間
            (check, time_data) = self.__to_interval_check(target, time_data)
            if check:
                time_data.interval =True
                return time_data
                   
        return time_data
    
    def __this_week_interval_check(self, target, time_data):
        """
        本週
        :return:
        """
        
        check = False
        rule = u"(本周|本週|本星期|本禮拜)([一二兩三四五六日])?"
        pattern = re.compile(rule)         
        match = pattern.search(target)

        if match is not None:
            
            check = True
            ## 本週 
            if match.group(2) is not None:
                return check, time_data

            dt1 = datetime.strptime(time_data.time_list[0], self.time_format)
#                 dt1 = dt1.replace(hour=0, minute=0, second=0) 
            dt2 = dt1 + timedelta(days=(7))

            if dt1>dt2:
                dt1,dt2=dt2,dt1

            time_data.time_list = [dt1.strftime(self.time_format),dt2.strftime(self.time_format)]        

#             time_data.time_type = 'interval'
            
        return  (check, time_data)
    
    def __to_interval_check(self, target, time_data):
        """
        判斷區間
        :return:
        """
        
        check = False
        rule = u"(到|至)"
        pattern = re.compile(rule)
        match = pattern.search(target)

        if match is not None:
#             time_data.time_type = 'interval'

            check = True

        return (check, time_data)
    
    def __past_interval_check(self, target, time_data):
        
        check = False
        rule = u"(過去)?([一二兩三四五六七八九1-9])+(個)?(周|週|星期|禮拜)"
        pattern = re.compile(rule)         
        match = pattern.search(target)            

        if match is not None:

            dt1 = datetime.strptime(time_data.time_list[0], self.time_format)
#             dt1 = dt1.replace(hour=0, minute=0, second=0)
#             dt2 = datetime.strptime(time[len(time) - 1], self.time_format)
            if match.group(1) == u"過去":

                dt2 = dt1 - timedelta(days=(7 * StringPreHandler.wordToNumber(match.group(2))))
            else:

                multiple =  StringPreHandler.wordToNumber(match.group(2))                   
                dt2 = dt1 + timedelta(days=(7 * multiple))

            if dt1>dt2:
                dt1,dt2=dt2,dt1

            time_data.time_list = [dt1.strftime(self.time_format),dt2.strftime(self.time_format)]         
#             time_data.time_type = 'interval'

            check = True
            
        return (check, time_data)

In [9]:
# print arrow.now().strftime('%Y-%m-%d %H:%M:%S'),print datetime.now().strftime('%Y-%m-%d %H:%M:%S')

In [10]:
from flask import Flask, jsonify
def get_time(target):
    
    if target == '':
        return ''
    
    target= target.decode('utf8')
    tn = TimeGenerator(isPreferFuture=False)
    result = tn.get_time(target)
    
    if not result:
        return ''

    print json.dumps(result, default=lambda o:o.__dict__)
    
    print '=========================='
    print 'repeat = ' + str(result.repeat)
    print 'interval = ' + str(result.interval)
#     print  result.time_list
    for item in result.time_interval_data_list:
        print '---------------------------------'
        print 'start_date = ' + item.start_date
        print 'start_time = ' + item.start_time
        print 'end_date = ' + item.end_date
        print 'end_time = ' + item.end_time

In [11]:
get_time('上午的天氣')
get_time('上班的天氣')
get_time('下班的天氣')
#     day_break = 6  # 凌晨
#     early_morning = 9  # 上班|早上|早晨|早間|晨間|今早|明早
#     morning = 10  # 上午
#     noon = 12  # (中午)|(午間)|(午休)
#     afternoon = 15  # (下午)|(午後)
#     night = 18  # 下班
#     lateNight = 20  # 晚上|夜間|夜裡|今晚|明晚
#     midNight = 23  # 深夜

{"interval": false, "time_interval_data_list": [{"start_time": "10:00:00", "start_date": "2018-06-05", "end_date": "", "end_time": ""}], "repeat": false}
repeat = False
interval = False
---------------------------------
start_date = 2018-06-05
start_time = 10:00:00
end_date = 
end_time = 
{"interval": false, "time_interval_data_list": [{"start_time": "09:00:00", "start_date": "2018-06-05", "end_date": "", "end_time": ""}], "repeat": false}
repeat = False
interval = False
---------------------------------
start_date = 2018-06-05
start_time = 09:00:00
end_date = 
end_time = 
{"interval": false, "time_interval_data_list": [{"start_time": "18:00:00", "start_date": "2018-06-05", "end_date": "", "end_time": ""}], "repeat": false}
repeat = False
interval = False
---------------------------------
start_date = 2018-06-05
start_time = 18:00:00
end_date = 
end_time = 


In [12]:
get_time('今天中午')
get_time('今天下午')
get_time('明天早上')

{"interval": false, "time_interval_data_list": [{"start_time": "12:00:00", "start_date": "2018-06-05", "end_date": "", "end_time": ""}], "repeat": false}
repeat = False
interval = False
---------------------------------
start_date = 2018-06-05
start_time = 12:00:00
end_date = 
end_time = 
{"interval": false, "time_interval_data_list": [{"start_time": "15:00:00", "start_date": "2018-06-05", "end_date": "", "end_time": ""}], "repeat": false}
repeat = False
interval = False
---------------------------------
start_date = 2018-06-05
start_time = 15:00:00
end_date = 
end_time = 
{"interval": false, "time_interval_data_list": [{"start_time": "09:00:00", "start_date": "2018-06-06", "end_date": "", "end_time": ""}], "repeat": false}
repeat = False
interval = False
---------------------------------
start_date = 2018-06-06
start_time = 09:00:00
end_date = 
end_time = 


In [13]:
get_time('2018年2月2日早上或2018年2月20日下午八點三十分二十九秒')

{"interval": false, "time_interval_data_list": [{"start_time": "09:00:00", "start_date": "2018-02-02", "end_date": "", "end_time": ""}, {"start_time": "20:30:29", "start_date": "2018-02-20", "end_date": "", "end_time": ""}], "repeat": false}
repeat = False
interval = False
---------------------------------
start_date = 2018-02-02
start_time = 09:00:00
end_date = 
end_time = 
---------------------------------
start_date = 2018-02-20
start_time = 20:30:29
end_date = 
end_time = 


In [14]:
get_time('2018年2月28日下午三點,2018年3月5日早上八點20')

{"interval": false, "time_interval_data_list": [{"start_time": "15:00:00", "start_date": "2018-02-28", "end_date": "", "end_time": ""}, {"start_time": "08:20:00", "start_date": "2018-03-05", "end_date": "", "end_time": ""}], "repeat": false}
repeat = False
interval = False
---------------------------------
start_date = 2018-02-28
start_time = 15:00:00
end_date = 
end_time = 
---------------------------------
start_date = 2018-03-05
start_time = 08:20:00
end_date = 
end_time = 


In [15]:
get_time('2018年2月28日下午三點')

{"interval": false, "time_interval_data_list": [{"start_time": "15:00:00", "start_date": "2018-02-28", "end_date": "", "end_time": ""}], "repeat": false}
repeat = False
interval = False
---------------------------------
start_date = 2018-02-28
start_time = 15:00:00
end_date = 
end_time = 


In [16]:
get_time('今年4月1日3點')

{"interval": false, "time_interval_data_list": [{"start_time": "03:00:00", "start_date": "2018-04-01", "end_date": "", "end_time": ""}], "repeat": false}
repeat = False
interval = False
---------------------------------
start_date = 2018-04-01
start_time = 03:00:00
end_date = 
end_time = 


In [17]:
get_time('下禮拜四下午')

{"interval": false, "time_interval_data_list": [{"start_time": "15:00:00", "start_date": "2018-06-14", "end_date": "", "end_time": ""}], "repeat": false}
repeat = False
interval = False
---------------------------------
start_date = 2018-06-14
start_time = 15:00:00
end_date = 
end_time = 


In [18]:
get_time('下星期二早上4點50分20到8點2分1秒')

{"interval": true, "time_interval_data_list": [{"start_time": "04:50:20", "start_date": "2018-06-12", "end_date": "2018-06-12", "end_time": "08:02:01"}], "repeat": false}
repeat = False
interval = True
---------------------------------
start_date = 2018-06-12
start_time = 04:50:20
end_date = 2018-06-12
end_time = 08:02:01


In [19]:
get_time('7夕')

{"interval": false, "time_interval_data_list": [{"start_time": "00:00:00", "start_date": "2018-08-17", "end_date": "", "end_time": ""}], "repeat": false}
repeat = False
interval = False
---------------------------------
start_date = 2018-08-17
start_time = 00:00:00
end_date = 
end_time = 


In [20]:
get_time('雙十節')

{"interval": false, "time_interval_data_list": [{"start_time": "00:00:00", "start_date": "2018-10-10", "end_date": "", "end_time": ""}], "repeat": false}
repeat = False
interval = False
---------------------------------
start_date = 2018-10-10
start_time = 00:00:00
end_date = 
end_time = 


In [21]:
get_time('明年春節')

{"interval": false, "time_interval_data_list": [{"start_time": "00:00:00", "start_date": "2019-02-05", "end_date": "", "end_time": ""}], "repeat": false}
repeat = False
interval = False
---------------------------------
start_date = 2019-02-05
start_time = 00:00:00
end_date = 
end_time = 


In [22]:
get_time('隔天')

{"interval": false, "time_interval_data_list": [{"start_time": "00:00:00", "start_date": "2018-06-06", "end_date": "", "end_time": ""}], "repeat": false}
repeat = False
interval = False
---------------------------------
start_date = 2018-06-06
start_time = 00:00:00
end_date = 
end_time = 


In [23]:
get_time('明天')

{"interval": false, "time_interval_data_list": [{"start_time": "00:00:00", "start_date": "2018-06-06", "end_date": "", "end_time": ""}], "repeat": false}
repeat = False
interval = False
---------------------------------
start_date = 2018-06-06
start_time = 00:00:00
end_date = 
end_time = 


In [24]:
get_time('明天的後天')

{"interval": false, "time_interval_data_list": [{"start_time": "00:00:00", "start_date": "2018-06-08", "end_date": "", "end_time": ""}], "repeat": false}
repeat = False
interval = False
---------------------------------
start_date = 2018-06-08
start_time = 00:00:00
end_date = 
end_time = 


In [25]:
#fail
get_time('明天的明天的明天')

{"interval": false, "time_interval_data_list": [{"start_time": "00:00:00", "start_date": "2018-06-06", "end_date": "", "end_time": ""}], "repeat": false}
repeat = False
interval = False
---------------------------------
start_date = 2018-06-06
start_time = 00:00:00
end_date = 
end_time = 


In [26]:
get_time('明天到後天')

{"interval": true, "time_interval_data_list": [{"start_time": "00:00:00", "start_date": "2018-06-06", "end_date": "2018-06-07", "end_time": "00:00:00"}], "repeat": false}
repeat = False
interval = True
---------------------------------
start_date = 2018-06-06
start_time = 00:00:00
end_date = 2018-06-07
end_time = 00:00:00


In [27]:
get_time('2020年4月1日到4月10日下午')

{"interval": true, "time_interval_data_list": [{"start_time": "00:00:00", "start_date": "2020-04-01", "end_date": "2020-04-10", "end_time": "15:00:00"}], "repeat": false}
repeat = False
interval = True
---------------------------------
start_date = 2020-04-01
start_time = 00:00:00
end_date = 2020-04-10
end_time = 15:00:00


In [28]:
get_time('兩天之前到四天之後')

{"interval": true, "time_interval_data_list": [{"start_time": "00:00:00", "start_date": "2018-06-03", "end_date": "2018-06-09", "end_time": "00:00:00"}], "repeat": false}
repeat = False
interval = True
---------------------------------
start_date = 2018-06-03
start_time = 00:00:00
end_date = 2018-06-09
end_time = 00:00:00


In [29]:
get_time('後天十三點20分26秒到15點19')

{"interval": true, "time_interval_data_list": [{"start_time": "13:20:26", "start_date": "2018-06-07", "end_date": "2018-06-07", "end_time": "15:19:00"}], "repeat": false}
repeat = False
interval = True
---------------------------------
start_date = 2018-06-07
start_time = 13:20:26
end_date = 2018-06-07
end_time = 15:19:00


In [30]:
##禮拜日爲最後一天
get_time('禮拜日')

{"interval": false, "time_interval_data_list": [{"start_time": "00:00:00", "start_date": "2018-06-10", "end_date": "", "end_time": ""}], "repeat": false}
repeat = False
interval = False
---------------------------------
start_date = 2018-06-10
start_time = 00:00:00
end_date = 
end_time = 


In [31]:
get_time('禮拜一')

{"interval": false, "time_interval_data_list": [{"start_time": "00:00:00", "start_date": "2018-06-04", "end_date": "", "end_time": ""}], "repeat": false}
repeat = False
interval = False
---------------------------------
start_date = 2018-06-04
start_time = 00:00:00
end_date = 
end_time = 


In [32]:
get_time('禮拜天早上到禮拜三下午')

{"interval": true, "time_interval_data_list": [{"start_time": "09:00:00", "start_date": "2018-06-10", "end_date": "2018-06-06", "end_time": "15:00:00"}], "repeat": false}
repeat = False
interval = True
---------------------------------
start_date = 2018-06-10
start_time = 09:00:00
end_date = 2018-06-06
end_time = 15:00:00


In [33]:
get_time('週末')

{"interval": false, "time_interval_data_list": [{"start_time": "00:00:00", "start_date": "2018-06-10", "end_date": "", "end_time": ""}], "repeat": false}
repeat = False
interval = False
---------------------------------
start_date = 2018-06-10
start_time = 00:00:00
end_date = 
end_time = 


In [34]:
get_time('過去兩周')

{"interval": true, "time_interval_data_list": [{"start_time": "16:08:50", "start_date": "2018-05-22", "end_date": "2018-06-05", "end_time": "16:08:50"}], "repeat": false}
repeat = False
interval = True
---------------------------------
start_date = 2018-05-22
start_time = 16:08:50
end_date = 2018-06-05
end_time = 16:08:50


In [35]:
get_time('2月之後')

{"interval": false, "time_interval_data_list": [{"start_time": "00:00:00", "start_date": "2018-08-01", "end_date": "", "end_time": ""}], "repeat": false}
repeat = False
interval = False
---------------------------------
start_date = 2018-08-01
start_time = 00:00:00
end_date = 
end_time = 


In [36]:
get_time('')

''

In [37]:
get_time('天氣')

''

In [38]:
get_time('本周')

{"interval": true, "time_interval_data_list": [{"start_time": "16:08:50", "start_date": "2018-06-05", "end_date": "2018-06-12", "end_time": "16:08:50"}], "repeat": false}
repeat = False
interval = True
---------------------------------
start_date = 2018-06-05
start_time = 16:08:50
end_date = 2018-06-12
end_time = 16:08:50


In [39]:
### get_time('周日')
get_time('一周下午三點')

{"interval": true, "time_interval_data_list": [{"start_time": "16:08:50", "start_date": "2018-06-05", "end_date": "2018-06-12", "end_time": "16:08:50"}], "repeat": false}
repeat = False
interval = True
---------------------------------
start_date = 2018-06-05
start_time = 16:08:50
end_date = 2018-06-12
end_time = 16:08:50


In [40]:
get_time('每周二下午三點到5點')
# get_time('每周下午三點20分')

{"interval": true, "time_interval_data_list": [{"start_time": "15:00:00", "start_date": "2018-06-05", "end_date": "2018-06-05", "end_time": "17:00:00"}], "repeat": true}
repeat = True
interval = True
---------------------------------
start_date = 2018-06-05
start_time = 15:00:00
end_date = 2018-06-05
end_time = 17:00:00


In [41]:
get_time('每周二下午兩點到五點與周四')

{"interval": true, "time_interval_data_list": [{"start_time": "14:00:00", "start_date": "2018-06-05", "end_date": "2018-06-07", "end_time": "00:00:00"}], "repeat": true}
repeat = True
interval = True
---------------------------------
start_date = 2018-06-05
start_time = 14:00:00
end_date = 2018-06-07
end_time = 00:00:00


In [42]:
get_time('下下週')

{"interval": false, "time_interval_data_list": [{"start_time": "00:00:00", "start_date": "2018-06-18", "end_date": "", "end_time": ""}], "repeat": false}
repeat = False
interval = False
---------------------------------
start_date = 2018-06-18
start_time = 00:00:00
end_date = 
end_time = 


In [43]:
get_time('每週二到週四')

{"interval": true, "time_interval_data_list": [{"start_time": "00:00:00", "start_date": "2018-06-05", "end_date": "2018-06-07", "end_time": "00:00:00"}], "repeat": true}
repeat = True
interval = True
---------------------------------
start_date = 2018-06-05
start_time = 00:00:00
end_date = 2018-06-07
end_time = 00:00:00


In [44]:
#fail
get_time('週三,週二到周四')

{"interval": true, "time_interval_data_list": [{"start_time": "00:00:00", "start_date": "2018-06-06", "end_date": "2018-06-07", "end_time": "00:00:00"}], "repeat": false}
repeat = False
interval = True
---------------------------------
start_date = 2018-06-06
start_time = 00:00:00
end_date = 2018-06-07
end_time = 00:00:00


In [48]:
get_time('明天下午三點到後天下午五點')

{"interval": true, "time_interval_data_list": [{"start_time": "15:00:00", "start_date": "2018-06-06", "end_date": "2018-06-07", "end_time": "17:00:00"}], "repeat": false}
repeat = False
interval = True
---------------------------------
start_date = 2018-06-06
start_time = 15:00:00
end_date = 2018-06-07
end_time = 17:00:00
